In [ ]:
import numpy as np
import pandas as pd
import math

pd.options.display.max_columns = 100

In [ ]:
import matplotlib.pyplot as plt

%matplotlib inline
%config InlineBackend.figure_format = 'svg'

In [ ]:
data = pd.read_csv('input/train.csv')

In [ ]:
# data = data[data['HouseYear'] != 1977]

In [ ]:
s = (data['Floor'] > data['HouseFloor']) | (data['HouseFloor'] > 40)
data.loc[s, ['Floor','HouseFloor']] = math.floor(data['Floor'].median()), math.floor(data['HouseFloor'].median())

In [ ]:
data.shape

In [ ]:
data.head()

In [ ]:
data.loc[data['Rooms'] > 5, :]
data.loc[data['Id'].isin([5927, 14003, 14865]), 'Rooms'] = 2
data.loc[data['Id'] == 8491, 'Rooms'] = 1

data.loc[data['HouseYear'] == 20052011, 'HouseYear'] = 2008
data.loc[data['HouseYear'] == 4968, 'HouseYear'] = 1968

small_square_ids = data.loc[data['Square'] < 15, 'Id'].values
mean_square = data.groupby('Rooms', as_index=False)[['Square']].mean().rename(columns={'Square': 'mean_square'})
data = pd.merge(data, mean_square, on='Rooms', how='left')
data.loc[data['Id'].isin(small_square_ids), 'Square'] = data['mean_square']
data.loc[data['Id'].isin(small_square_ids), 'LifeSquare'] = data['Square']

large_square_ids = data.loc[data['Square'] > 150, 'Id'].values
data.loc[data['Id'].isin(large_square_ids), 'Square'] = data['mean_square']
data.loc[data['Id'].isin(large_square_ids), 'LifeSquare'] = data['Square']

data['LifeSquare'] = data['LifeSquare'].fillna(data['Square'])

data = data.drop(columns='mean_square')

### train_test_split

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
train, valid = train_test_split(data, test_size=0.3, random_state=45)

In [ ]:
train.shape, valid.shape

### Prepare train, valid

In [ ]:
def add_info1(df, df_source, feat):
    flat_qty = 'flat_qty_'+feat
    distr_info1 = df_source[feat].value_counts(normalize=True).reset_index().\
        rename(columns={'index':feat, feat:flat_qty})
    
    df = pd.merge(df, distr_info1, on=feat, how='left')
    df[flat_qty] = df[flat_qty].fillna(df[flat_qty].min())
    return df

In [ ]:
def add_mean(df, df_source, feat):
    m_feat = 'mean_' + feat
    
    distr_stat_dr = df_source.groupby([feat, 'Rooms'], as_index=False)[['Price']].mean().\
        rename(columns={'Price':m_feat})
    distr_stat_r = df_source.groupby(['Rooms'], as_index=False)[['Price']].mean().rename(columns={'Price':'mean_r'})
    m_price = df_source['Price'].median()
    
    df = pd.merge(df, distr_stat_dr, on=[feat, 'Rooms'], how='left')
    df = pd.merge(df, distr_stat_r, on='Rooms', how='left')
    
    df['mean_r'] = df['mean_r'].fillna(m_price)
    df[m_feat] = df[m_feat].fillna(df['mean_r'])
    df = df.drop(columns='mean_r')
    return df

In [ ]:
def add_median(df, df_source, feat):
    m_feat = 'mdn_' + feat
    
    distr_stat_dr = df_source.groupby([feat, 'Rooms'], as_index=False)[['Price']].median().\
        rename(columns={'Price':m_feat})
    distr_stat_r = df_source.groupby(['Rooms'], as_index=False)[['Price']].median().rename(columns={'Price':'mdn_r'})
    m_price = df_source['Price'].median()
    
    df = pd.merge(df, distr_stat_dr, on=[feat, 'Rooms'], how='left')
    df = pd.merge(df, distr_stat_r, on='Rooms', how='left')
    
    df['mdn_r'] = df['mdn_r'].fillna(m_price)
    df[m_feat] = df[m_feat].fillna(df['mdn_r'])
    df = df.drop(columns='mdn_r')
    return df

In [ ]:
def add_cat_fts(df, cat_fts=('Ecology_2', 'Ecology_3', 'Shops_2')):
    for col in cat_fts:
        df[col] = (df[col] == 'B').astype(int)
    return df

In [ ]:
def fillna_healthcare_1(df):
    df['Healthcare_1'] = df['Healthcare_1'].fillna(0)
    return df

In [ ]:
def prepare_data(df, df_source):
    df = add_info1(df, df_source, 'DistrictId')
    df = add_info1(df, df_source, 'HouseFloor')
    df = add_info1(df, df_source, 'HouseYear')
    df = add_info1(df, df_source, 'Social_3')
    df = add_info1(df, df_source, 'Helthcare_2')
    df = add_info1(df, df_source, 'Shops_1')
    
    df = add_mean(df, df_source, 'DistrictId')
    df = add_mean(df, df_source, 'HouseFloor')
    df = add_mean(df, df_source, 'HouseYear')
    df = add_mean(df, df_source, 'Social_3')
    df = add_mean(df, df_source, 'Helthcare_2')
    df = add_mean(df, df_source, 'Shops_1')
    
    df = add_median(df, df_source, 'DistrictId')
    df = add_median(df, df_source, 'HouseFloor')
    df = add_median(df, df_source, 'HouseYear')
    df = add_median(df, df_source, 'Social_3')
    df = add_median(df, df_source, 'Helthcare_2')
    df = add_median(df, df_source, 'Shops_1')
    
    df = add_cat_fts(df)
    df = fillna_healthcare_1(df)
    
    return df

In [ ]:
train = prepare_data(train, train)

In [ ]:
train.head()

In [ ]:
train.info()

In [ ]:
valid = prepare_data(valid, train)

In [ ]:
feats1 = list(filter(lambda s: s[0:5] != 'mean_' and s[0:4] != 'mdn_', train.columns))
data1 = pd.DataFrame(train, columns=feats1)

data1.hist(bins=40, figsize=(8, 5), zorder=2)
plt.tight_layout()
plt.xlabel('', fontsize=4)
plt.xlabel('', fontsize=4);

In [ ]:
feats2 = list(filter(lambda s: s[0:5] == 'mean_', train.columns))
data2 = pd.DataFrame(train, columns=feats2)
data2.hist(bins=40, figsize=(10, 3), zorder=2)
plt.tight_layout()
plt.xlabel('', fontsize=4);

In [ ]:
feats3 = list(filter(lambda s: s[0:4] == 'mdn_', train.columns))
data3 = pd.DataFrame(train, columns=feats3)
data3.hist(bins=40, figsize=(10, 3), zorder=2)
plt.tight_layout()
plt.xlabel('', fontsize=4);

In [ ]:
feat = 'HouseFloor'
year = 1977

min_year = np.int(data['HouseYear'].min())
max_year = np.int(data['HouseYear'].max())
display((min_year, max_year))

for year in np.sort(data['HouseYear'].unique()):
    if year < 1964:
        continue
    data_year = data[data['HouseYear'] == year]
    data_year_2 = data[(data['HouseYear'] == year) & (data['Floor'] > data['HouseFloor'])]
    plt.scatter(data_year[feat], data_year['Price'], zorder=2);
    plt.scatter(data_year_2[feat], data_year_2['Price'], zorder=2);
    
    min_hf = np.int(data[data['HouseYear'] == year][feat].min())
    max_hf = np.int(data[data['HouseYear'] == year][feat].max() + 0.99)
    
    plt.xticks(range(min_hf, max_hf))
    plt.gca().xaxis.grid(True)
    plt.gcf().set_size_inches(11, 3)
    plt.title(year)
    plt.show()

### Coef

In [ ]:
feats = train.columns.drop(['Price', 'LifeSquare', 'Id', 'DistrictId', 'HouseFloor', 'HouseYear', 'Social_3', 'Helthcare_2', 'Shops_1']).values
feats

In [ ]:
from sklearn.linear_model import LinearRegression as LR
from sklearn.preprocessing import StandardScaler

In [ ]:
lr = LR()
scaler = StandardScaler()

train_scaled = pd.DataFrame(train, columns=feats)
train_scaled = scaler.fit_transform(train_scaled)

# valid_scaled = pd.DataFrame(valid, columns=feats)
# valid_scaled = scaler.transform(valid_scaled)

lr.fit(train_scaled, train['Price'])

In [ ]:
plt.rcParams.update({'font.size': 7})
plt.rcParams["axes.grid.axis"] = "y"
plt.rcParams["axes.grid"] = True

plt.barh(feats, lr.coef_.flatten(), zorder=2)
plt.xlabel('Вес признака')
plt.ylabel('Признак')

fig = plt.gcf()
fig.set_size_inches(11.2, 4)
plt.gca().invert_yaxis()

### Model

In [ ]:
train.columns

In [ ]:
# feats = ['Rooms', 'Square', 'flat_qty_DistrictId', 'mean_DistrictId', 'Helthcare_2', 'Healthcare_1', 'HouseYear']
#feats = ['Id', 'Square', 'Floor', 'HouseFloor', 'Ecology_3', 'Social_1', 'Social_2', 'Healthcare_1', 'Shops_1', 'Price']

In [ ]:
from sklearn.ensemble import RandomForestRegressor as RF

In [ ]:
# ?RF

In [ ]:
model = RF(n_estimators=300, max_depth=12, random_state=42, max_features=4, min_samples_leaf=2)

In [ ]:
train.info()

In [ ]:
model.fit(train.loc[:, feats], train['Price'])

In [ ]:
pred_train = model.predict(train.loc[:, feats])

In [ ]:
pred_train.shape

In [ ]:
pred_train

In [ ]:
pred_valid = model.predict(valid.loc[:, feats])

In [ ]:
pred_valid.shape

In [ ]:
pred_valid

### Evaluate model

In [ ]:
from sklearn.metrics import r2_score as r2

In [ ]:
r2(train['Price'], pred_train)

In [ ]:
r2(valid['Price'], pred_valid)

In [ ]:
feats

### Test

In [ ]:
# ?prepare_data

In [ ]:
test = pd.read_csv('input/test.csv')

In [ ]:
test = prepare_data(test, train)

In [ ]:
test.loc[:, feats].info()

In [ ]:
test['Price'] = model.predict(test.loc[:, feats])

In [ ]:
test.loc[:, ['Id', 'Price']].to_csv('AAnonymous_predictions.csv', index=None)

In [ ]:
import random

In [ ]:
# columns = list(train.columns)
# columns.remove('Id')
# #columns.remove('LifeSquare')
# columns.remove('Price')

In [ ]:
from itertools import compress

def filt(feat_genes):
    return list(compress(feats, feat_genes))

In [ ]:
# len(train.columns)

In [ ]:
feat_genes = [random.choice([True, False]) for _ in range(len(feats))]
feat_genes

In [ ]:
train.head()

In [ ]:
len(train.columns)

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
parameters = [{'n_estimators': [20],
               'max_features': np.arange(len(train.columns) - 2, len(train.columns) - 1),
               'max_depth': [12],
               'min_samples_leaf': [2]}]

In [ ]:
gscv = GridSearchCV(estimator=RF(random_state=150),
                    param_grid=parameters,
                    scoring='r2',
                    cv=5,
                    verbose=2,
                    n_jobs=-1)

In [ ]:
%%time
gscv.fit(train.drop(columns='Price'), train['Price'])

In [ ]:
means = gscv.cv_results_['mean_test_score']
stds = gscv.cv_results_['std_test_score']

for mean, std, params in zip(means, stds, gscv.cv_results_['params']):
    print('%0.3f (+/-%0.03f) for %r' % (mean, std * 2, params))

In [ ]:
gscv.best_params_

In [ ]:
rf2 = RF(n_estimators=300, #gscv.best_params_['n_estimators'],
         max_features=gscv.best_params_['max_features'],
         max_depth=gscv.best_params_['max_depth'],
         min_samples_leaf=gscv.best_params_['min_samples_leaf'],
         
         random_state=130)

# rf2 = RF(max_depth=10,
#          max_features=18,
#          n_estimators=300,
#          random_state=130)

#rf2 = RF(n_estimators=300, max_depth=12, random_state=42, max_features=4, min_samples_leaf=2)

In [ ]:
rf2.fit(train.drop(columns='Price'), train['Price'])

In [ ]:
pred_train = rf2.predict(train.drop(columns='Price'))

In [ ]:
pred_train

In [ ]:
pred_valid = rf2.predict(valid.drop(columns='Price'))

In [ ]:
r2(train['Price'], pred_train)

In [ ]:
r2(valid['Price'], pred_valid)

In [ ]:
model = RF(n_estimators=300, max_depth=12, random_state=46, max_features=4, min_samples_leaf=2)
model.fit(train.drop(columns='Price'), train['Price'])
pred_train = model.predict(train.drop(columns='Price'))
pred_valid = model.predict(valid.drop(columns='Price'))

In [ ]:
r2(train['Price'], pred_train)

In [ ]:
r2(valid['Price'], pred_valid)

### Genetic

In [ ]:
feats

In [ ]:
import random
from concurrent.futures import ProcessPoolExecutor

class Chromosome:
    def __init__(self, generate=False):
        if generate:
            self._generate()
        self.fitness = 0
        self.done = False
        self.just_done = False
        self.mutated = False
        
    @property
    def feats_genes(self):
        return self.genes[:len(feats)]

    @property
    def other_genes(self):
        return self.genes[len(feats):]

    def _generate(self):
        self.genes = [random.choice([True, False]) for _ in range(len(feats))]

        n_estimators = 100
        max_depth = random.randint(8, 18)
        min_samples_split = random.randint(1, 10)
        min_samples_leaf = random.randint(1, 10)
        min_weight_fraction_leaf = random.uniform(0, 5)
        max_features = random.choice([None, 'sqrt', 'log2', random.randint(3, len(feats))])
        max_leaf_nodes = random.randint(1, 100)
        min_impurity_decrease = random.uniform(0, 5)
        
        self.genes.append(n_estimators)
        self.genes.append(max_depth)
        self.genes.append(min_samples_split)
        self.genes.append(min_samples_leaf)
        self.genes.append(min_weight_fraction_leaf)
        self.genes.append(max_features)
        self.genes.append(max_leaf_nodes)
        self.genes.append(min_impurity_decrease)

    def set_fitness(self):
        if self.done:
            self.just_done = False
        else:
            feats_genes = filt(self.genes)
            try:
                model = RF(n_estimators=self.other_genes[0],
                        max_depth = self.other_genes[1],
                        min_samples_leaf = self.other_genes[3],
                        max_features = self.other_genes[5],
                        random_state=42,
                        n_jobs=-1)
                r2s = []
                for i in range(len(trains)):
                    model.fit(trains[i].loc[:, feats_genes], np.array(trains[i]['Price']))
                    pred_valid = model.predict(valids[i].loc[:, feats_genes])
                    r2_ = r2(valids[i]['Price'], pred_valid)
                    r2s.append(r2_)
                self.fitness = sum(r2s) / len(r2s)
            except:
                self.fitness = 0
            self.done = self.just_done = True

    @property
    def feats_genes_str(self):
       ints = [int(b) for b in self.feats_genes]
       # return ints.__str__()
       return '[' + ''.join(map(str, ints)) + ']'

    @property
    def other_genes_str(self):
        return ['{0:0.2f}'.format(i) if type(i) is float else i for i in self.other_genes]
    
    def __str__(self):
        just_done = '*' if self.just_done else ' '
        mutated = 'm' if self.mutated else ' '
        return f'{100*self.fitness:6.2f}% {self.feats_genes_str} {self.other_genes_str} {just_done} {mutated}'


def cross(chro1, chro2):
    genes_len = len(chro1.genes)
    r1 = random.randint(0, genes_len - 1)
    r2 = random.randint(r1 + 1, genes_len)

    child1 = Chromosome()
    child2 = Chromosome()

    child1.genes = chro1.genes[:r1] + chro2.genes[r1:r2] + chro1.genes[r2:]
    child2.genes = chro2.genes[:r1] + chro1.genes[r1:r2] + chro2.genes[r2:]

    return child1, child2


def ftns(chro):
    print('---')
    chro.set_fitness()

class Ga:
    def __init__(self, size):
        self.generate(size)

    def generate(self, size):
        self.generation = [Chromosome(True) for _ in range(size)]
        self.set_fitnesses()
        self._sort()
        self.iteration = 1

    def next(self):
        end = len(self.generation)
        middle = end // 2
        gener = self.generation
        for i in range(middle, end, 2):
            parent1, parent2 = self._random_choice(middle)
            gener[i], gener[i + 1] = cross(gener[parent1], gener[parent2])
        self.set_fitnesses()
        self._sort()
        self.iteration += 1

    def set_fitnesses(self):
        j = 0
        for chro in self.generation:
            chro.set_fitness()
            j += 1
            print(j, end=' ')
        print()
        
    def _random_choice(self, n):
        x = [i for i in range(n)]
        prob = [i for i in range(n, 0, -1)]
        sum_ = sum(prob)
        for i in range(len(prob)):
            prob[i] /= sum_
        return np.random.choice(n, 2, replace=False)
            
            
    def _sort(self):
        self.generation = sorted(self.generation, key=lambda x: x.fitness, reverse=True)

    def __str__(self):
        ret = f'================== {self.iteration:3} ==================\n'
        output_top = 50
        if len(self.generation) < output_top:
            output_top = len(self.generation)

        for i in range(output_top):
            ret += f'{(i + 1): 3})'
            ret += self.generation[i].__str__()
            ret += '\n'
        return ret


# POPULATION = 128
# ga = Ga(POPULATION)
# print(ga)

# for i in range(1, 100000):
#     ga.next()
#     print(ga)

==================   1 ==================
  1) 71.51% [11111001000100100000001011100] [100, 15, 2, 4, '1.96', 'log2', 79, '2.86'] *  
  2) 70.94% [01100011010101111010100001001] [100, 18, 6, 3, '4.43', None, 78, '1.75'] *  
  3) 70.72% [01011101011111010010011010011] [100, 14, 3, 2, '2.26', 3, 17, '4.47'] *  
  4) 70.61% [11101111110111101001011001010] [100, 16, 10, 6, '4.52', None, 90, '2.75'] *  
  5) 70.40% [01001111100100100111010010110] [100, 14, 2, 1, '4.25', 'log2', 45, '1.03'] *  
  6) 70.25% [11100000100100010011111000111] [100, 16, 2, 4, '2.07', 'sqrt', 70, '4.37'] *  
  7) 70.12% [11101110110100000111011010001] [100, 17, 6, 1, '4.42', 'sqrt', 28, '2.62'] *  
  8) 70.09% [11100101100011101100101000011] [100, 10, 7, 1, '4.51', 'sqrt', 6, '3.61'] *  
  9) 70.08% [01011010111000101101011011000] [100, 15, 4, 2, '4.53', 'log2', 46, '3.82'] *  
 10) 70.01% [11110000011101100100111001001] [100, 16, 4, 4, '4.82', 'sqrt', 78, '1.02'] *  
 11) 69.74% [11000111110101001010000101011] [100, 15, 6, 2, '2.01', 7, 75, '1.47'] *  
 12) 69.71% [11101000100000010111010010011] [100, 18, 1, 1, '4.70', 'log2', 73, '3.57'] *  
 13) 69.68% [01101001110000100010001010011] [100, 12, 10, 8, '3.52', None, 62, '2.53'] *  
 14) 69.67% [11111000010110101000110111000] [100, 18, 6, 8, '4.17', 'sqrt', 3, '0.26'] *  
 15) 69.66% [01110001100001000101101000001] [100, 14, 10, 1, '3.53', 'log2', 75, '4.19'] *  
 16) 69.58% [01111111000110100111000001001] [100, 17, 10, 7, '1.40', 'sqrt', 58, '2.43'] *  
 17) 69.18% [01001101101001101100100010010] [100, 14, 2, 9, '3.61', 'log2', 3, '0.70'] *  
 18) 69.00% [11011011000011100111111101111] [100, 18, 10, 1, '4.87', 'sqrt', 81, '1.27'] *  
 19) 68.97% [11010101000000010000111001110] [100, 13, 2, 2, '3.22', 'log2', 19, '4.45'] *  
 20) 68.93% [01111111101111011001111111011] [100, 14, 6, 4, '0.18', None, 30, '2.57'] *  
 21) 68.70% [11101000110101111100001000001] [100, 17, 2, 2, '4.29', None, 86, '2.24'] *  
 22) 68.69% [01000001011101001111001110001] [100, 13, 3, 3, '1.23', 'sqrt', 88, '4.00'] *  
 23) 68.64% [01000101001111100001010110100] [100, 17, 9, 10, '3.08', 'log2', 2, '1.05'] *  
 24) 68.55% [11000010100001100110010001010] [100, 15, 1, 3, '2.82', None, 81, '4.09'] *  
 25) 68.52% [11010001101101111101011001110] [100, 15, 3, 1, '1.46', None, 67, '0.08'] *  
 26) 68.51% [11010101111110010100010000101] [100, 11, 6, 8, '3.07', 'log2', 43, '2.18'] *  
 27) 68.50% [01000101100001011101000011111] [100, 18, 6, 5, '1.31', 8, 93, '0.42'] *  
 28) 68.39% [01010010111010101101010010100] [100, 16, 7, 10, '1.99', 'log2', 15, '0.71'] *  
 29) 68.34% [01010100000100011000110111101] [100, 9, 7, 2, '0.11', 'sqrt', 61, '2.62'] *  
 30) 68.32% [01101010110111110100111110000] [100, 16, 1, 10, '0.71', 3, 78, '3.80'] *  
 31) 68.24% [01101001011011000111011010000] [100, 13, 3, 2, '1.60', None, 52, '3.16'] *  
 32) 68.20% [01110000110010110110010011110] [100, 10, 8, 3, '1.71', None, 25, '0.03'] *  
 33) 68.20% [11000110001101100011100001000] [100, 18, 7, 7, '4.02', None, 8, '2.64'] *  
 34) 68.14% [11011100011101110111101100111] [100, 11, 7, 5, '3.40', 'sqrt', 86, '0.70'] *  
 35) 68.10% [01001101101010100000011101010] [100, 16, 8, 9, '3.54', None, 32, '0.82'] *  
 36) 68.09% [11111110100111110101111110010] [100, 9, 4, 1, '4.54', None, 42, '2.53'] *  
 37) 68.08% [11000100100100101100000011100] [100, 14, 2, 8, '3.16', None, 17, '2.55'] *  
 38) 68.02% [11001011010111000001110001010] [100, 16, 8, 9, '2.41', 'sqrt', 51, '2.90'] *  
 39) 67.98% [01101010111011010111010101100] [100, 18, 9, 7, '0.78', None, 20, '0.97'] *  
 40) 67.93% [01111110001010110110110111101] [100, 18, 8, 8, '0.28', 'sqrt', 81, '3.81'] *  
 41) 67.89% [11010001001101110110111100001] [100, 17, 10, 8, '2.05', 'log2', 45, '3.03'] *  
 42) 67.86% [11011110111111000011000100001] [100, 18, 3, 10, '0.23', None, 92, '0.82'] *  
 43) 67.85% [11000010111001010001110010101] [100, 15, 2, 7, '2.80', None, 94, '2.00'] *  
 44) 67.80% [11001100101011101100110100110] [100, 17, 7, 8, '2.56', 'sqrt', 51, '2.65'] *  
 45) 67.77% [01101100010011011111010100101] [100, 16, 8, 8, '3.22', 'log2', 14, '1.84'] *  
 46) 67.74% [11011010011011111100011101001] [100, 18, 7, 9, '1.67', 'log2', 6, '3.64'] *  
 47) 67.68% [01110100011100101100111111100] [100, 9, 3, 6, '1.88', 'log2', 42, '1.80'] *  
 48) 67.65% [01110011000000101010001010101] [100, 16, 3, 9, '2.35', None, 71, '3.35'] *  
 49) 67.59% [01110010010110000100100110110] [100, 18, 3, 8, '1.59', 'sqrt', 81, '1.71'] *  
 50) 67.55% [01011110000011101101100001011] [100, 12, 3, 10, '1.34', None, 33, '3.88'] *  

==================   2 ==================
  1) 71.51% [11111001000100100000001011100] [100, 15, 2, 4, '1.96', 'log2', 79, '2.86']    
  2) 70.94% [01100011010101111010100001001] [100, 18, 6, 3, '4.43', None, 78, '1.75']    
  3) 70.72% [01011101011111010010011010011] [100, 14, 3, 2, '2.26', 3, 17, '4.47']    
  4) 70.61% [11101111110111101001011001010] [100, 16, 10, 6, '4.52', None, 90, '2.75']    
  5) 70.40% [01001111100100100111010010110] [100, 14, 2, 1, '4.25', 'log2', 45, '1.03']    
  6) 70.40% [01001111100100100111010010110] [100, 14, 2, 1, '4.25', 'log2', 45, '1.03'] *  
  7) 70.25% [11100000100100010011111000111] [100, 16, 2, 4, '2.07', 'sqrt', 70, '4.37']    
  8) 70.20% [01001111100100100111010010110] [100, 18, 1, 1, '4.70', 'log2', 45, '1.03'] *  
  9) 70.12% [11101110110100000111011010001] [100, 17, 6, 1, '4.42', 'sqrt', 28, '2.62']    
 10) 70.09% [11100101100011101100101000011] [100, 10, 7, 1, '4.51', 'sqrt', 6, '3.61']    
 11) 70.08% [01011010111000101101011011000] [100, 15, 4, 2, '4.53', 'log2', 46, '3.82']    
 12) 70.03% [01001110111000101101011011000] [100, 15, 4, 2, '4.53', 'log2', 46, '0.70'] *  
 13) 70.03% [11011110111111000001110010101] [100, 15, 2, 7, '2.80', None, 92, '0.82'] *  
 14) 70.01% [11110000011101100100111001001] [100, 16, 4, 4, '4.82', 'sqrt', 78, '1.02']    
 15) 69.96% [11110101101001100100111001001] [100, 16, 4, 4, '4.82', 'sqrt', 78, '1.02'] *  
 16) 69.77% [01000001011101001001001110001] [100, 13, 3, 3, '1.23', 'sqrt', 88, '4.00'] *  
 17) 69.74% [11000111110101001010000101011] [100, 15, 6, 2, '2.01', 7, 75, '1.47']    
 18) 69.74% [11000111110000010000111001110] [100, 13, 2, 2, '3.22', 'log2', 19, '4.45'] *  
 19) 69.71% [11101000100000010111010010011] [100, 18, 1, 1, '4.70', 'log2', 73, '3.57']    
 20) 69.69% [11101000100000010111010010011] [100, 14, 2, 1, '4.25', 'log2', 73, '3.57'] *  
 21) 69.68% [01101001110000100010001010011] [100, 12, 10, 8, '3.52', None, 62, '2.53']    
 22) 69.68% [01101001110000100010001010011] [100, 12, 10, 8, '3.52', None, 62, '2.53'] *  
 23) 69.67% [11111000010110101000110111000] [100, 18, 6, 8, '4.17', 'sqrt', 3, '0.26']    
 24) 69.66% [11111000010110101000110111000] [100, 12, 10, 8, '4.17', 'sqrt', 3, '0.26'] *  
 25) 69.66% [01110001100001000101101000001] [100, 14, 10, 1, '3.53', 'log2', 75, '4.19']    
 26) 69.65% [11100101100011101100101000011] [100, 11, 7, 5, '4.51', 'sqrt', 6, '3.61'] *  
 27) 69.63% [01101010110111110000011101010] [100, 16, 8, 9, '0.71', 3, 78, '3.80'] *  
 28) 69.59% [01111111000110100111000001010] [100, 17, 10, 7, '1.40', 'sqrt', 58, '2.43'] *  
 29) 69.58% [01111111000110100111000001001] [100, 17, 10, 7, '1.40', 'sqrt', 58, '2.43']    
 30) 69.47% [11000111110101001010000100101] [100, 11, 6, 2, '2.01', 7, 75, '1.47'] *  
 31) 69.36% [01101001110000100010001010011] [100, 10, 8, 8, '3.52', None, 62, '2.53'] *  
 32) 69.32% [11010101000101001010000101011] [100, 15, 6, 2, '2.01', 7, 75, '1.47'] *  
 33) 69.25% [01010010111101100011100001000] [100, 18, 7, 7, '4.02', None, 8, '2.64'] *  
 34) 69.24% [01011001101001101100100010010] [100, 14, 2, 9, '3.61', 'log2', 3, '3.82'] *  
 35) 69.19% [11011001110001100111111101111] [100, 18, 10, 1, '4.87', 'sqrt', 81, '1.27'] *  
 36) 69.18% [01001101101001101100100010010] [100, 14, 2, 9, '3.61', 'log2', 3, '0.70']    
 37) 69.18% [01001101101001101100100010010] [100, 14, 2, 9, '3.61', 'log2', 3, '0.70'] *  
 38) 69.18% [11010001101101100111111101111] [100, 18, 10, 1, '4.87', 'sqrt', 67, '0.08'] *  
 39) 69.15% [01010010111010101100000101011] [100, 15, 6, 2, '2.01', 'log2', 15, '0.71'] *  
 40) 69.11% [01001101101001101110001010010] [100, 14, 2, 9, '3.61', 'log2', 3, '0.70'] *  
 41) 69.11% [01101011000010100010001010011] [100, 12, 10, 8, '3.52', None, 62, '2.53'] *  
 42) 69.04% [01001000011101101100100010010] [100, 14, 2, 9, '3.61', 'log2', 3, '0.70'] *  
 43) 69.02% [11000111110101001011010010100] [100, 16, 7, 10, '1.99', 7, 75, '1.47'] *  
 44) 69.00% [11011011000011100111111101111] [100, 18, 10, 1, '4.87', 'sqrt', 81, '1.27']    
 45) 69.00% [01011101011111010010011010011] [100, 8, 4, 3, '4.81', None, 85, '1.55'] *  
 46) 68.97% [11010101000000010000111001110] [100, 13, 2, 2, '3.22', 'log2', 19, '4.45']    
 47) 68.93% [11010101111110010100010001011] [100, 15, 6, 8, '3.07', 'log2', 43, '2.18'] *  
 48) 68.93% [01000000110010110110010011111] [100, 18, 6, 5, '1.31', 8, 93, '0.42'] *  
 49) 68.93% [11001011001011011010100001011] [100, 18, 5, 5, '2.24', 'sqrt', 70, '1.85'] *  
 50) 68.93% [01111111101111011001111111011] [100, 14, 6, 4, '0.18', None, 30, '2.57']    

==================   3 ==================
  1) 71.51% [11111001000100100000001011100] [100, 15, 2, 4, '1.96', 'log2', 79, '2.86']    
  2) 71.41% [11111001000100100000001011100] [100, 15, 2, 4, '1.96', None, 86, '2.86'] *  
  3) 70.94% [01100011010101111010100001001] [100, 18, 6, 3, '4.43', None, 78, '1.75']    
  4) 70.78% [11101111100100100111010010110] [100, 14, 2, 1, '4.25', 'log2', 73, '3.57'] *  
  5) 70.72% [01011101011111010010011010011] [100, 14, 3, 2, '2.26', 3, 17, '4.47'] *  
  6) 70.72% [01011101011111010010011010011] [100, 14, 3, 2, '2.26', 3, 17, '4.47']    
  7) 70.72% [11101000110101111100001000001] [100, 17, 2, 2, '4.29', 'log2', 79, '2.24'] *  
  8) 70.61% [11101111110111101001011001010] [100, 16, 10, 6, '4.52', None, 90, '2.75']    
  9) 70.61% [11101111110111101001011001010] [100, 16, 10, 6, '4.52', None, 85, '2.75'] *  
 10) 70.50% [11011011000010100010001101111] [100, 18, 10, 1, '4.87', 'sqrt', 81, '1.27'] *  
 11) 70.40% [01001111100100100111010010110] [100, 14, 2, 1, '4.25', 'log2', 45, '1.03']    
 12) 70.40% [01001111100100100111010010110] [100, 14, 2, 1, '4.25', 'log2', 45, '1.03']    
 13) 70.40% [01001111100100100111010010110] [100, 14, 2, 1, '4.25', 'log2', 45, '0.70'] *  
 14) 70.32% [11100011010101111010100100111] [100, 10, 7, 1, '3.40', 'sqrt', 86, '0.70'] *  
 15) 70.31% [11111000010110101000110110100] [100, 17, 1, 1, '3.08', 'log2', 2, '1.05'] *  
 16) 70.25% [11100000100100010011111000111] [100, 16, 2, 4, '2.07', 'sqrt', 70, '4.37']    
 17) 70.24% [01011010111000101101011010010] [100, 15, 4, 2, '4.53', 'log2', 46, '3.82'] *  
 18) 70.20% [11011101011111100100111001001] [100, 16, 4, 4, '4.82', 'sqrt', 78, '1.02'] *  
 19) 70.20% [01001111100100100111010010110] [100, 18, 1, 1, '4.70', 'log2', 45, '1.03']    
 20) 70.20% [01001111100100100111010010110] [100, 18, 1, 1, '4.70', 'log2', 45, '1.03'] *  
 21) 70.20% [01001111100100100111010010110] [100, 18, 1, 1, '4.70', 'log2', 45, '0.70'] *  
 22) 70.17% [11011001110001100110101000011] [100, 18, 10, 1, '4.87', 'sqrt', 81, '1.27'] *  
 23) 70.12% [11101110110100000111011010001] [100, 17, 6, 1, '4.42', 'sqrt', 28, '2.62']    
 24) 70.09% [11100101100011101100101000011] [100, 10, 7, 1, '4.51', 'sqrt', 17, '4.47'] *  
 25) 70.09% [11100101100011101100101000011] [100, 10, 7, 1, '4.51', 'sqrt', 6, '3.61']    
 26) 70.08% [01011010111000101101011011000] [100, 15, 4, 2, '4.53', 'log2', 46, '3.82']    
 27) 70.08% [01011010111000101101011011000] [100, 15, 3, 2, '4.53', 'log2', 46, '3.82'] *  
 28) 70.03% [01001110111000101101011011000] [100, 15, 4, 2, '4.53', 'log2', 46, '0.70']    
 29) 70.03% [11011110111111000001110010101] [100, 15, 2, 7, '2.80', None, 92, '0.82']    
 30) 70.01% [11110000011101100100111001001] [100, 16, 4, 4, '4.82', 'sqrt', 78, '1.02']    
 31) 70.01% [01101001110000100010001010011] [100, 11, 7, 5, '3.52', None, 62, '2.53'] *  
 32) 69.97% [11101000110000010111010010011] [100, 18, 1, 1, '4.70', 'log2', 73, '3.57'] *  
 33) 69.96% [11110101101001100100111001001] [100, 16, 4, 4, '4.82', 'sqrt', 78, '1.02']    
 34) 69.95% [01001110111000101101011011000] [100, 18, 10, 1, '4.53', 'log2', 46, '0.70'] *  
 35) 69.95% [01001111100100100111010010110] [100, 11, 2, 1, '4.25', 'log2', 45, '1.03'] *  
 36) 69.91% [01011010111000101011111011000] [100, 15, 4, 2, '4.53', 'log2', 46, '3.82'] *  
 37) 69.88% [11100101100011101100101000011] [100, 14, 7, 5, '4.51', 'sqrt', 6, '3.61'] *  
 38) 69.79% [11000111110101001010001111011] [100, 14, 6, 2, '2.01', 7, 75, '1.47'] *  
 39) 69.77% [01000001011101001001001110001] [100, 13, 3, 3, '1.23', 'sqrt', 88, '4.00']    
 40) 69.74% [01111111101111011001111111011] [100, 18, 5, 5, '2.24', 'sqrt', 30, '2.57'] *  
 41) 69.74% [11000111110101001010000101011] [100, 15, 6, 2, '2.01', 7, 75, '1.47']    
 42) 69.74% [11000111110000010000111001110] [100, 13, 2, 2, '3.22', 'log2', 19, '4.45']    
 43) 69.71% [11101000100000010111010010011] [100, 18, 1, 1, '4.70', 'log2', 73, '3.57']    
 44) 69.69% [11101000100000010111010010011] [100, 14, 2, 1, '4.25', 'log2', 73, '3.57']    
 45) 69.68% [01101001110000100010001010011] [100, 12, 10, 8, '3.52', None, 62, '2.53']    
 46) 69.68% [01101001110000100010001010011] [100, 12, 10, 8, '3.52', None, 62, '2.53']    
 47) 69.67% [11111000010110101000110111000] [100, 18, 6, 8, '4.17', 'sqrt', 3, '0.26']    
 48) 69.66% [11111000010110101000110111000] [100, 12, 10, 8, '4.17', 'sqrt', 3, '0.26']    
 49) 69.66% [01110001100001000101101000001] [100, 14, 10, 1, '3.53', 'log2', 75, '4.19']    
 50) 69.65% [11100101100011101100101000011] [100, 11, 7, 5, '4.51', 'sqrt', 6, '3.61']    

==================   4 ==================
  1) 72.40% [11101110110100111010100001010] [100, 16, 8, 1, '4.42', 'sqrt', 28, '2.62'] *  
  2) 71.60% [11001011010101111010100001010] [100, 16, 2, 1, '4.25', 'log2', 45, '2.90'] *  
  3) 71.51% [11111001000100100000001011100] [100, 15, 2, 4, '1.96', 'log2', 79, '2.86']    
  4) 71.41% [11111001000100100000001011100] [100, 15, 2, 4, '1.96', None, 86, '2.86']    
  5) 71.41% [11111001000100100000001011100] [100, 15, 2, 4, '1.96', None, 86, '2.86'] *  
  6) 71.39% [01100011010101111010100001110] [100, 18, 1, 1, '4.70', 'log2', 45, '1.75'] *  
  7) 71.13% [11100011010101111011010010011] [100, 14, 2, 1, '4.25', 'log2', 73, '0.70'] *  
  8) 70.94% [01100011010101111010100001001] [100, 18, 6, 3, '4.43', None, 78, '1.75']    
  9) 70.94% [11111001000100000001110010101] [100, 15, 2, 4, '1.96', None, 86, '2.86'] *  
 10) 70.87% [11100011010101111010111001001] [100, 16, 4, 4, '4.82', 'sqrt', 78, '0.70'] *  
 11) 70.86% [01100011110101001010100001001] [100, 18, 6, 3, '4.43', None, 78, '1.75'] *  
 12) 70.80% [11011110111111000001110010101] [100, 14, 3, 2, '2.26', 3, 17, '0.82'] *  
 13) 70.78% [11101111100100100111010010110] [100, 14, 2, 1, '4.25', 'log2', 73, '3.57']    
 14) 70.78% [11101111100100100111010010110] [100, 14, 2, 1, '4.25', 'sqrt', 6, '1.27'] *  
 15) 70.78% [11101111100100100111010010110] [100, 14, 2, 1, '4.25', 'log2', 73, '3.57'] *  
 16) 70.76% [11101000100100100111010010011] [100, 14, 2, 1, '4.25', 'log2', 73, '3.57'] *  
 17) 70.72% [01011101011111010010011010011] [100, 14, 3, 2, '2.26', 3, 17, '4.47']    
 18) 70.72% [01011101011111010010011010011] [100, 14, 3, 2, '2.26', 3, 17, '4.47']    
 19) 70.72% [11101000110101111100001000001] [100, 17, 2, 2, '4.29', 'log2', 79, '2.24']    
 20) 70.64% [11011011000010100010000001001] [100, 17, 10, 1, '4.87', 'sqrt', 81, '1.27'] *  
 21) 70.64% [11111000110101111100001000001] [100, 17, 4, 4, '4.82', 'sqrt', 78, '1.02'] *  
 22) 70.61% [11101111110111101001011001010] [100, 16, 10, 6, '4.52', None, 90, '2.75']    
 23) 70.61% [11101111110111101001011001010] [100, 16, 10, 6, '4.52', None, 85, '2.75']    
 24) 70.61% [01111111101110100010001101011] [100, 18, 5, 5, '2.24', 'sqrt', 30, '2.57'] *  
 25) 70.50% [11011011000010100010001101111] [100, 18, 10, 1, '4.87', 'sqrt', 81, '1.27']    
 26) 70.49% [11011110111111100000001011100] [100, 15, 2, 7, '2.80', None, 92, '0.82'] *  
 27) 70.47% [11000001000100100000001011100] [100, 15, 2, 4, '2.01', 7, 75, '1.47'] *  
 28) 70.40% [01001111100100100111010010110] [100, 14, 2, 1, '4.25', 'log2', 45, '1.03']    
 29) 70.40% [01001111100100100111010010110] [100, 14, 2, 1, '4.25', 'log2', 45, '1.03']    
 30) 70.40% [01001111100100100111010010110] [100, 14, 2, 1, '4.25', 'log2', 45, '0.70']    
 31) 70.40% [01001111100100100111010010110] [100, 14, 2, 1, '4.25', 'log2', 45, '4.19'] *  
 32) 70.39% [01011101011111010010011010011] [100, 15, 2, 7, '2.80', None, 92, '4.47'] *  
 33) 70.38% [11101110111000101111011010001] [100, 17, 6, 1, '4.42', 'sqrt', 28, '2.62'] *  
 34) 70.36% [11100101101001100100111001001] [100, 16, 2, 2, '4.29', 'log2', 79, '2.24'] *  
 35) 70.32% [11100011010101111010100100111] [100, 10, 7, 1, '3.40', 'sqrt', 86, '0.70']    
 36) 70.31% [11111000010110101000110110100] [100, 17, 1, 1, '3.08', 'log2', 2, '1.05']    
 37) 70.29% [11101000110101111100001000001] [100, 18, 1, 1, '4.70', 'log2', 73, '3.57'] *  
 38) 70.28% [11101000100011101100101000011] [100, 18, 1, 1, '4.70', 'log2', 73, '3.57'] *  
 39) 70.27% [11100011010111110000100100111] [100, 10, 7, 1, '3.40', 'sqrt', 86, '0.70'] *  
 40) 70.25% [11100000100100010011111000111] [100, 16, 2, 4, '2.07', 'sqrt', 70, '4.37']    
 41) 70.25% [11100000100100010011111000111] [100, 16, 2, 4, '2.07', 'sqrt', 70, '4.37'] *  
 42) 70.24% [01011010111000101101011010010] [100, 15, 4, 2, '4.53', 'log2', 46, '3.82']    
 43) 70.20% [11011101011111100100111001001] [100, 16, 4, 4, '4.82', 'sqrt', 78, '1.02']    
 44) 70.20% [01001111100100100111010010110] [100, 18, 1, 1, '4.70', 'log2', 45, '1.03']    
 45) 70.20% [01001111100100100111010010110] [100, 18, 1, 1, '4.70', 'log2', 45, '1.03']    
 46) 70.20% [01001111100100100111010010110] [100, 18, 1, 1, '4.70', 'log2', 45, '0.70']    
 47) 70.20% [11000111010101111010001111011] [100, 14, 6, 2, '2.01', 7, 75, '1.47'] *  
 48) 70.18% [01011010111000101101011011000] [100, 17, 10, 2, '4.53', 'log2', 46, '3.82'] *  
 49) 70.18% [11001111100100100111010010110] [100, 14, 2, 1, '4.25', 'log2', 73, '3.57'] *  
 50) 70.17% [11011001110001100110101000011] [100, 18, 10, 1, '4.87', 'sqrt', 81, '1.27']    

==================   5 ==================
  1) 72.40% [11101110110100111010100001010] [100, 16, 8, 1, '4.42', 'sqrt', 28, '2.62']    
  2) 72.30% [11111001000100100000001011100] [100, 18, 1, 1, '4.70', 'log2', 73, '3.57'] *  
  3) 72.25% [11111001000100100000001011100] [100, 15, 2, 1, '4.87', 'sqrt', 86, '2.86'] *  
  4) 71.99% [01101111110111101001010010110] [100, 14, 2, 1, '4.25', 'log2', 45, '0.70'] *  
  5) 71.82% [11001011010101111010100001010] [100, 14, 2, 1, '4.25', 'sqrt', 45, '2.90'] *  
  6) 71.74% [11101011110101001010100001001] [100, 18, 6, 3, '4.43', 'log2', 79, '2.24'] *  
  7) 71.60% [11001011010101111010100001010] [100, 16, 2, 1, '4.25', 'log2', 45, '2.90']    
  8) 71.51% [11111001000100100000001011100] [100, 15, 2, 4, '1.96', 'log2', 79, '2.86']    
  9) 71.41% [11111001000100100000001011100] [100, 15, 2, 4, '1.96', None, 86, '2.86']    
 10) 71.41% [11111001000100100000001011100] [100, 15, 2, 4, '1.96', None, 86, '2.86']    
 11) 71.39% [01100011010101111010100001110] [100, 18, 1, 1, '4.70', 'log2', 45, '1.75']    
 12) 71.31% [11011110111111000001110010101] [100, 17, 2, 2, '4.29', 'log2', 17, '0.82'] *  
 13) 71.15% [11000001000100100000001011101] [100, 14, 2, 1, '4.25', 'log2', 73, '3.57'] *  
 14) 71.13% [11100011010101111011010010011] [100, 14, 2, 1, '4.25', 'log2', 73, '0.70']    
 15) 71.04% [11100011010101111011010010110] [100, 14, 2, 1, '4.25', 'log2', 78, '0.70'] *  
 16) 70.94% [01100011010101111010100001001] [100, 18, 6, 3, '4.43', None, 78, '1.75']    
 17) 70.94% [11111001000100000001110010101] [100, 15, 2, 4, '1.96', None, 86, '2.86']    
 18) 70.92% [11100001000100100000001011100] [100, 15, 2, 4, '2.01', 7, 75, '1.47'] *  
 19) 70.92% [11100011010101111010100100011] [100, 14, 2, 1, '3.40', 'sqrt', 86, '0.70'] *  
 20) 70.89% [11100011010101111010011010011] [100, 15, 2, 4, '4.82', 'sqrt', 78, '0.70'] *  
 21) 70.87% [11100011010101111010111001001] [100, 16, 4, 4, '4.82', 'sqrt', 78, '0.70']    
 22) 70.86% [01100011110101001010100001001] [100, 18, 6, 3, '4.43', None, 78, '1.75']    
 23) 70.82% [11101000110101111100001001001] [100, 18, 1, 1, '4.70', 'log2', 73, '3.57'] *  
 24) 70.81% [11100011010111110000100100011] [100, 18, 1, 1, '3.40', 'sqrt', 86, '0.70'] *  
 25) 70.80% [11011110111111000001110010101] [100, 14, 3, 2, '2.26', 3, 17, '0.82']    
 26) 70.78% [11101111100100100111010010110] [100, 14, 2, 1, '4.25', 'log2', 73, '3.57']    
 27) 70.78% [11101111100100100111010010110] [100, 14, 2, 1, '4.25', 'sqrt', 6, '1.27']    
 28) 70.78% [11101111100100100111010010110] [100, 14, 2, 1, '4.25', 'log2', 73, '3.57']    
 29) 70.78% [11100011000100100000001011100] [100, 15, 2, 4, '2.01', 7, 86, '0.70'] *  
 30) 70.76% [11101000100100100111010010011] [100, 14, 2, 1, '4.25', 'log2', 73, '3.57']    
 31) 70.72% [01011101011111010010011010011] [100, 14, 3, 2, '2.26', 3, 17, '4.47']    
 32) 70.72% [01011101011111010010011010011] [100, 14, 3, 2, '2.26', 3, 17, '4.47']    
 33) 70.72% [11101000110101111100001000001] [100, 17, 2, 2, '4.29', 'log2', 79, '2.24']    
 34) 70.68% [11111001000100000001110010101] [100, 15, 2, 4, '1.96', 'log2', 46, '3.82'] *  
 35) 70.67% [11100011010101111010111011001] [100, 16, 4, 4, '4.82', 'sqrt', 78, '0.70'] *  
 36) 70.65% [11101111100100100111010010110] [100, 16, 2, 1, '4.25', 'log2', 6, '1.27'] *  
 37) 70.64% [11011011000010100010000001001] [100, 17, 10, 1, '4.87', 'sqrt', 81, '1.27']    
 38) 70.64% [11111000110101111100001000001] [100, 17, 4, 4, '4.82', 'sqrt', 78, '1.02']    
 39) 70.62% [11111000110101111100001001100] [100, 15, 4, 4, '4.82', 'sqrt', 78, '1.02'] *  
 40) 70.61% [11100101100011101100001011100] [100, 15, 7, 1, '4.51', 'sqrt', 17, '4.47'] *  
 41) 70.61% [11101111110111101001011001010] [100, 16, 10, 6, '4.52', None, 90, '2.75']    
 42) 70.61% [11101111110111101001011001010] [100, 16, 10, 6, '4.52', None, 85, '2.75']    
 43) 70.61% [11101000100100100111010010011] [100, 17, 10, 2, '4.25', 'log2', 73, '3.57'] *  
 44) 70.61% [01111111101110100010001101011] [100, 18, 5, 5, '2.24', 'sqrt', 30, '2.57']    
 45) 70.58% [01001111100100100010001101111] [100, 18, 10, 1, '4.87', 'sqrt', 81, '1.27'] *  
 46) 70.58% [01011101011111010011011010001] [100, 17, 6, 1, '4.42', 'sqrt', 28, '2.62'] *  
 47) 70.52% [11111001000100100000101000011] [100, 10, 2, 4, '1.96', None, 86, '2.86'] *  
 48) 70.50% [11011011000010100010001101111] [100, 18, 10, 1, '4.87', 'sqrt', 81, '1.27']    
 49) 70.49% [11011110111111100000001011100] [100, 15, 2, 7, '2.80', None, 92, '0.82']    
 50) 70.47% [11000001000100100000001011100] [100, 15, 2, 4, '2.01', 7, 75, '1.47']    

==================   6 ==================
  1) 72.40% [11101110110100111010100001010] [100, 16, 8, 1, '4.42', 'sqrt', 28, '2.62']    
  2) 72.30% [11111001000100100000001011100] [100, 18, 1, 1, '4.70', 'log2', 73, '3.57']    
  3) 72.25% [11111001000100100000001011100] [100, 15, 2, 1, '4.87', 'sqrt', 86, '2.86']    
  4) 72.25% [11111001000100100000001011100] [100, 15, 2, 1, '4.87', 'sqrt', 86, '2.86'] *  
  5) 72.18% [11111001000100100000001011111] [100, 18, 1, 1, '4.70', 'log2', 73, '3.57'] *  
  6) 72.16% [11101001000100100000001001001] [100, 18, 1, 1, '4.70', 'log2', 73, '3.57'] *  
  7) 71.99% [01101111110111101001010010110] [100, 14, 2, 1, '4.25', 'log2', 45, '0.70']    
  8) 71.82% [11001011010101111010100001010] [100, 14, 2, 1, '4.25', 'sqrt', 45, '2.90']    
  9) 71.78% [11011110110100111010100001010] [100, 16, 8, 1, '4.42', 'sqrt', 28, '2.62'] *  
 10) 71.78% [11011101011111100000001011100] [100, 18, 1, 1, '4.70', 'log2', 73, '3.57'] *  
 11) 71.74% [11101011110101001010100001001] [100, 18, 6, 3, '4.43', 'log2', 79, '2.24']    
 12) 71.68% [11101110110100111011010010110] [100, 14, 2, 1, '4.25', 'log2', 28, '2.62'] *  
 13) 71.68% [11100011010101111010100001010] [100, 16, 8, 1, '4.42', 'sqrt', 78, '0.70'] *  
 14) 71.65% [11111000110101101010001000001] [100, 17, 4, 4, '4.82', 'sqrt', 78, '1.02'] *  
 15) 71.61% [11100011010101111010101001011] [100, 14, 2, 1, '3.40', 'sqrt', 86, '0.70'] *  
 16) 71.60% [11001011010101111010100001010] [100, 16, 2, 1, '4.25', 'log2', 45, '2.90']    
 17) 71.54% [01001111100111100000001011100] [100, 15, 2, 2, '4.29', 'log2', 92, '0.82'] *  
 18) 71.51% [11111001000100100000001011100] [100, 15, 2, 4, '1.96', 'log2', 79, '2.86']    
 19) 71.50% [01101111110111000001110010110] [100, 14, 2, 1, '4.25', 'log2', 45, '0.70'] *  
 20) 71.48% [01100110111101111010100001110] [100, 18, 1, 1, '4.70', 'log2', 45, '1.75'] *  
 21) 71.47% [11100011010101111010011010011] [100, 17, 2, 2, '4.29', 'log2', 78, '0.70'] *  
 22) 71.41% [11111001000100100000001011100] [100, 15, 2, 4, '1.96', None, 86, '2.86']    
 23) 71.41% [11111001000100100000001011100] [100, 15, 2, 4, '1.96', None, 86, '2.86']    
 24) 71.39% [01100011010101111010100001110] [100, 18, 1, 1, '4.70', 'log2', 45, '1.75']    
 25) 71.39% [01100011010101111010100001110] [100, 18, 1, 1, '4.70', 'log2', 45, '0.70'] *  
 26) 71.35% [01011101011101001010100001011] [100, 14, 3, 2, '2.26', 3, 17, '4.47'] *  
 27) 71.33% [11011110111111100000001010110] [100, 14, 2, 1, '4.25', None, 92, '0.82'] *  
 28) 71.31% [11011110111111000001110010101] [100, 17, 2, 2, '4.29', 'log2', 17, '0.82']    
 29) 71.31% [11011110111111000001110010101] [100, 14, 2, 1, '4.25', 'log2', 45, '0.82'] *  
 30) 71.24% [11111001000100000001110011100] [100, 18, 1, 1, '4.70', 'log2', 73, '3.57'] *  
 31) 71.23% [11011110111111101001010010101] [100, 14, 3, 2, '2.26', 3, 17, '0.82'] *  
 32) 71.15% [11000001000100100000001011101] [100, 14, 2, 1, '4.25', 'log2', 73, '3.57']    
 33) 71.14% [11101011110111010010011010001] [100, 18, 6, 3, '4.43', 'log2', 79, '2.24'] *  
 34) 71.13% [11100011010101111011010010011] [100, 14, 2, 1, '4.25', 'log2', 73, '0.70']    
 35) 71.06% [11011110111111000001110010101] [100, 14, 2, 2, '4.29', 'log2', 79, '0.82'] *  
 36) 71.05% [11111001000100100000001010101] [100, 15, 2, 4, '1.96', 'log2', 46, '3.82'] *  
 37) 71.05% [01100011010111101001011001010] [100, 16, 10, 1, '4.70', 'log2', 45, '1.75'] *  
 38) 71.04% [11100011010101111011010010110] [100, 14, 2, 1, '4.25', 'log2', 78, '0.70']    
 39) 71.04% [11100011010101111011010010110] [100, 14, 2, 1, '4.25', 'log2', 78, '1.75'] *  
 40) 70.99% [01011101011111010011011010110] [100, 14, 6, 1, '4.42', 'sqrt', 28, '2.62'] *  
 41) 70.96% [11101111110101111010100001110] [100, 18, 1, 6, '4.52', None, 85, '2.75'] *  
 42) 70.95% [11101111110111101000001011010] [100, 16, 10, 6, '4.52', None, 90, '2.75'] *  
 43) 70.94% [01100011010101111010100001001] [100, 18, 6, 3, '4.43', None, 78, '1.75']    
 44) 70.94% [11111001000100000001110010101] [100, 15, 2, 4, '1.96', None, 86, '2.86']    
 45) 70.92% [11111111101110100010011010011] [100, 14, 3, 2, '2.26', 3, 17, '4.47'] *  
 46) 70.92% [11100001000100100000001011100] [100, 15, 2, 4, '2.01', 7, 75, '1.47']    
 47) 70.92% [11100011010101111010100100011] [100, 14, 2, 1, '3.40', 'sqrt', 86, '0.70']    
 48) 70.89% [11100011010101111010011010011] [100, 15, 2, 4, '4.82', 'sqrt', 78, '0.70']    
 49) 70.87% [11100011010101111010111001001] [100, 16, 4, 4, '4.82', 'sqrt', 78, '0.70']    
 50) 70.86% [01100011110101001010100001001] [100, 18, 6, 3, '4.43', None, 78, '1.75']    

==================   7 ==================
  1) 72.40% [11101110110100111010100001010] [100, 16, 8, 1, '4.42', 'sqrt', 28, '2.62']    
  2) 72.32% [11111011101100100000001011111] [100, 18, 1, 1, '4.70', 'log2', 73, '3.57'] *  
  3) 72.30% [11111001000100100000001011100] [100, 18, 1, 1, '4.70', 'log2', 73, '3.57']    
  4) 72.28% [11101110110100100000001011100] [100, 18, 1, 1, '4.70', 'log2', 73, '3.57'] *  
  5) 72.25% [11111001000100100000001011100] [100, 15, 2, 1, '4.87', 'sqrt', 86, '2.86']    
  6) 72.25% [11111001000100100000001011100] [100, 15, 2, 1, '4.87', 'sqrt', 86, '2.86']    
  7) 72.18% [11111001000100100000001011111] [100, 18, 1, 1, '4.70', 'log2', 73, '3.57']    
  8) 72.16% [11101001000100100000001001001] [100, 18, 1, 1, '4.70', 'log2', 73, '3.57']    
  9) 72.16% [11101001000100100000001001001] [100, 18, 1, 1, '4.70', 'log2', 78, '3.57'] *  
 10) 71.99% [01101111110111101001010010110] [100, 14, 2, 1, '4.25', 'log2', 45, '0.70']    
 11) 71.88% [11011101011111111010100001010] [100, 16, 8, 1, '4.42', 'sqrt', 28, '2.62'] *  
 12) 71.82% [11101000110100100000001010001] [100, 17, 2, 2, '4.29', 'log2', 79, '2.24'] *  
 13) 71.82% [11001011010101111010100001010] [100, 14, 2, 1, '4.25', 'sqrt', 45, '2.90']    
 14) 71.79% [11101111110111101000001011100] [100, 15, 2, 4, '2.01', 7, 86, '0.70'] *  
 15) 71.78% [11011110110100111010100001010] [100, 16, 8, 1, '4.42', 'sqrt', 28, '2.62']    
 16) 71.78% [01100011010111101000001001110] [100, 18, 1, 1, '4.70', 'log2', 45, '0.70'] *  
 17) 71.78% [11101111110101001010100001001] [100, 18, 6, 3, '4.43', 'log2', 79, '2.24'] *  
 18) 71.78% [11011101011111100000001011100] [100, 18, 1, 1, '4.70', 'log2', 73, '3.57']    
 19) 71.77% [11101111110110100000001011100] [100, 15, 2, 4, '2.01', 7, 86, '2.75'] *  
 20) 71.76% [11100011010101111000001011111] [100, 18, 1, 1, '4.70', 'log2', 78, '0.70'] *  
 21) 71.74% [11101011110101001010100001001] [100, 18, 6, 3, '4.43', 'log2', 79, '2.24']    
 22) 71.72% [01101111110111101001010001001] [100, 18, 6, 1, '4.25', 'log2', 45, '0.70'] *  
 23) 71.68% [11101110110100111011010010110] [100, 14, 2, 1, '4.25', 'log2', 28, '2.62']    
 24) 71.68% [11100011010101111010100001010] [100, 16, 8, 1, '4.42', 'sqrt', 78, '0.70']    
 25) 71.65% [11111000110101101010001000001] [100, 17, 4, 4, '4.82', 'sqrt', 78, '1.02']    
 26) 71.61% [11100011010101111010101001011] [100, 14, 2, 1, '3.40', 'sqrt', 86, '0.70']    
 27) 71.60% [11001011010101111010100001010] [100, 16, 2, 1, '4.25', 'log2', 45, '2.90']    
 28) 71.60% [11001011010101111010100001010] [100, 16, 2, 1, '4.25', 'log2', 45, '2.90'] *  
 29) 71.59% [01101111110111101000000001001] [100, 18, 6, 3, '4.43', None, 78, '1.75'] *  
 30) 71.57% [11100011010111100000001010110] [100, 14, 2, 1, '4.25', 'log2', 73, '0.70'] *  
 31) 71.55% [11101011110111010010100010001] [100, 18, 6, 3, '4.43', 'log2', 79, '2.24'] *  
 32) 71.54% [01001111100111100000001011100] [100, 15, 2, 2, '4.29', 'log2', 92, '0.82']    
 33) 71.53% [11101111110111000001110010110] [100, 14, 2, 1, '4.25', 'log2', 45, '0.70'] *  
 34) 71.52% [01100011010101111010011001110] [100, 18, 1, 1, '4.70', 'log2', 45, '1.75'] *  
 35) 71.52% [01101011110111000001110010110] [100, 14, 2, 1, '4.25', 'log2', 45, '0.70'] *  
 36) 71.51% [11111001000100100000001011100] [100, 15, 2, 4, '1.96', 'log2', 79, '2.86']    
 37) 71.50% [01101111110111000001110010110] [100, 14, 2, 1, '4.25', 'log2', 45, '0.70']    
 38) 71.49% [11011101011111100000001011100] [100, 14, 3, 2, '2.26', 3, 17, '4.47'] *  
 39) 71.49% [11100011010101111010011010011] [100, 15, 2, 1, '4.42', 'sqrt', 28, '2.62'] *  
 40) 71.49% [11101111000100100000001011101] [100, 14, 2, 1, '4.52', None, 90, '2.75'] *  
 41) 71.49% [01011101010101111010100001011] [100, 14, 3, 2, '2.26', 3, 17, '4.47'] *  
 42) 71.48% [01100110111101111010100001110] [100, 18, 1, 1, '4.70', 'log2', 45, '1.75']    
 43) 71.47% [11100011010101111010011010011] [100, 17, 2, 2, '4.29', 'log2', 78, '0.70']    
 44) 71.47% [11100011010101111010011010011] [100, 17, 2, 2, '4.29', 'log2', 78, '0.70'] *  
 45) 71.45% [11101000110101110001110011001] [100, 18, 1, 1, '4.70', 'log2', 73, '3.57'] *  
 46) 71.45% [01101110111111000001110010101] [100, 14, 3, 2, '4.25', 'log2', 45, '0.70'] *  
 47) 71.45% [11101011110101001010100001001] [100, 18, 6, 3, '4.43', None, 79, '2.24'] *  
 48) 71.44% [01100011110101001010100001001] [100, 18, 6, 3, '4.43', 'log2', 78, '1.75'] *  
 49) 71.41% [11111001000100100000001011100] [100, 15, 2, 4, '1.96', None, 86, '2.86']    
 50) 71.41% [11111001000100100000001011100] [100, 15, 2, 4, '1.96', None, 86, '2.86']    

==================   8 ==================
  1) 72.63% [11101111110111101000000001001] [100, 18, 2, 2, '4.29', 'log2', 79, '2.24'] *  
  2) 72.40% [11101110110100111010100001010] [100, 16, 8, 1, '4.42', 'sqrt', 28, '2.62']    
  3) 72.32% [11111011101100100000001011111] [100, 18, 1, 1, '4.70', 'log2', 73, '3.57']    
  4) 72.30% [11111001000100100000001011100] [100, 18, 1, 1, '4.70', 'log2', 73, '3.57']    
  5) 72.28% [11101110110100100000001011100] [100, 18, 1, 1, '4.70', 'log2', 73, '3.57']    
  6) 72.25% [11111001000100100000001011100] [100, 15, 2, 1, '4.87', 'sqrt', 86, '2.86']    
  7) 72.25% [11111001000100100000001011100] [100, 15, 2, 1, '4.87', 'sqrt', 86, '2.86']    
  8) 72.24% [11111011010100100000001011111] [100, 18, 1, 1, '4.70', 'log2', 73, '3.57'] *  
  9) 72.18% [11111001000100100000001011111] [100, 18, 1, 1, '4.70', 'log2', 73, '3.57']    
 10) 72.16% [11101001000100100000001001001] [100, 18, 1, 1, '4.70', 'log2', 73, '3.57']    
 11) 72.16% [11101001000100100000001001001] [100, 18, 1, 1, '4.70', 'log2', 78, '3.57']    
 12) 72.12% [11101000110101110000110011001] [100, 18, 1, 1, '4.70', 'log2', 73, '3.57'] *  
 13) 72.11% [11111001000101111010100001010] [100, 14, 2, 1, '4.25', 'sqrt', 73, '3.57'] *  
 14) 72.07% [11111001000100111010100001011] [100, 18, 1, 1, '4.70', 'log2', 73, '3.57'] *  
 15) 72.04% [11111110110100111010100001010] [100, 18, 6, 3, '4.43', 'log2', 79, '2.24'] *  
 16) 71.99% [01101111110111101001010010110] [100, 14, 2, 1, '4.25', 'log2', 45, '0.70']    
 17) 71.90% [01100011010101111010100001010] [100, 16, 2, 1, '4.25', 'log2', 45, '2.90'] *  
 18) 71.89% [11100011010100111010100001010] [100, 16, 8, 1, '4.42', 'sqrt', 78, '0.70'] *  
 19) 71.88% [11011101011111111010100001010] [100, 16, 8, 1, '4.42', 'sqrt', 28, '2.62']    
 20) 71.88% [11011101011111111010100001010] [100, 16, 8, 1, '4.42', 'sqrt', 28, '2.62'] *  
 21) 71.82% [11101000110100100000001010001] [100, 17, 2, 2, '4.29', 'log2', 79, '2.24']    
 22) 71.82% [11001011010101111010100001010] [100, 14, 2, 1, '4.25', 'sqrt', 45, '2.90']    
 23) 71.82% [11001011010101111010100001010] [100, 14, 2, 1, '4.25', 'sqrt', 45, '1.75'] *  
 24) 71.82% [11011110110100100000001011110] [100, 16, 8, 1, '4.42', 'sqrt', 28, '2.62'] *  
 25) 71.79% [11101111110111101000001011100] [100, 15, 2, 4, '2.01', 7, 86, '0.70']    
 26) 71.79% [01011011010101111010010001110] [100, 18, 1, 1, '4.70', 3, 17, '4.47'] *  
 27) 71.79% [11001111000100100000001011101] [100, 14, 2, 1, '4.25', 'sqrt', 45, '2.90'] *  
 28) 71.78% [11011110110100111010100001010] [100, 16, 8, 1, '4.42', 'sqrt', 28, '2.62']    
 29) 71.78% [01100011010111101000001001110] [100, 18, 1, 1, '4.70', 'log2', 45, '0.70']    
 30) 71.78% [11101111110101001010100001001] [100, 18, 6, 3, '4.43', 'log2', 79, '2.24']    
 31) 71.78% [11101111110101001010100001001] [100, 18, 6, 3, '4.43', 'log2', 79, '0.70'] *  
 32) 71.78% [11011101011111100000001011100] [100, 18, 1, 1, '4.70', 'log2', 73, '3.57']    
 33) 71.77% [11101111110110100000001011100] [100, 15, 2, 4, '2.01', 7, 86, '2.75']    
 34) 71.77% [11100011010101101000011010011] [100, 17, 2, 2, '4.29', 'log2', 78, '0.70'] *  
 35) 71.76% [11100011010101111000001011111] [100, 18, 1, 1, '4.70', 'log2', 78, '0.70']    
 36) 71.76% [11100011010101111000001011111] [100, 18, 1, 1, '4.70', 'log2', 78, '2.24'] *  
 37) 71.76% [01100011010111100000001001110] [100, 18, 1, 1, '4.70', 'log2', 73, '3.57'] *  
 38) 71.74% [11101011110101001010100001001] [100, 18, 6, 3, '4.43', 'log2', 79, '2.24']    
 39) 71.72% [01101111110111101001010001001] [100, 14, 2, 1, '4.25', 'log2', 45, '0.70'] *  
 40) 71.72% [01101111110111101001010001001] [100, 18, 6, 1, '4.25', 'log2', 45, '0.70']    
 41) 71.71% [11001001000100100000001011110] [100, 16, 2, 1, '4.25', 'log2', 45, '2.90'] *  
 42) 71.71% [11011110111111100000001010110] [100, 14, 2, 1, '4.25', 'log2', 28, '2.62'] *  
 43) 71.71% [11101011110111010010100001001] [100, 18, 6, 3, '4.43', 'log2', 79, '2.24'] *  
 44) 71.68% [11101110110100111011010010110] [100, 14, 2, 1, '4.25', 'log2', 28, '2.62']    
 45) 71.68% [11101110110100111011010010110] [100, 14, 2, 1, '4.25', 'log2', 78, '2.62'] *  
 46) 71.68% [11100011010101111010100001010] [100, 16, 8, 1, '4.42', 'sqrt', 78, '0.70']    
 47) 71.68% [11001111110101001010100001001] [100, 16, 8, 1, '4.42', 'sqrt', 28, '2.62'] *  
 48) 71.67% [11001011010101111010100001000] [100, 16, 2, 1, '4.25', 'log2', 45, '2.90'] *  
 49) 71.65% [11111000110101101010001000001] [100, 17, 4, 4, '4.82', 'sqrt', 78, '1.02']    
 50) 71.63% [11111001000100100000001011101] [100, 14, 3, 2, '2.26', None, 86, '2.86'] *  

==================   9 ==================
  1) 72.63% [11101111110111101000000001001] [100, 18, 2, 2, '4.29', 'log2', 79, '2.24']    
  2) 72.58% [11111001000100100000000001000] [100, 15, 2, 1, '4.87', 'sqrt', 86, '2.86'] *  
  3) 72.44% [11101000110111101000000001001] [100, 18, 1, 1, '4.70', 'log2', 73, '3.57'] *  
  4) 72.40% [11101110110100111010100001010] [100, 16, 8, 1, '4.42', 'sqrt', 28, '2.62']    
  5) 72.32% [11111011101100100000001011111] [100, 18, 1, 1, '4.70', 'log2', 73, '3.57']    
  6) 72.30% [11111001000100100000001011100] [100, 18, 1, 1, '4.70', 'log2', 73, '3.57']    
  7) 72.30% [11111001000100100000001011100] [100, 18, 1, 1, '4.25', 'log2', 73, '3.57'] *  
  8) 72.29% [11101110110100111010100001010] [100, 14, 2, 1, '4.25', 'log2', 78, '2.62'] *  
  9) 72.28% [11101110110100100000001011100] [100, 18, 1, 1, '4.70', 'log2', 73, '3.57']    
 10) 72.28% [11111011010100100000001011110] [100, 18, 1, 1, '4.70', 'log2', 45, '3.57'] *  
 11) 72.25% [11111001000100100000001011100] [100, 15, 2, 1, '4.87', 'sqrt', 86, '2.86']    
 12) 72.25% [11111001000100100000001011100] [100, 15, 2, 1, '4.87', 'sqrt', 86, '2.86']    
 13) 72.25% [11101110110100100000001011101] [100, 14, 2, 1, '4.42', 'sqrt', 28, '2.62'] *  
 14) 72.24% [11111011010100100000001011111] [100, 18, 1, 1, '4.70', 'log2', 73, '3.57']    
 15) 72.24% [11111110110100111000100001010] [100, 18, 6, 3, '4.43', 'log2', 79, '2.24'] *  
 16) 72.21% [11101000110100111010100001001] [100, 17, 2, 2, '4.29', 'log2', 79, '2.24'] *  
 17) 72.19% [11101111110110100000001011100] [100, 18, 1, 1, '4.70', 'log2', 86, '2.75'] *  
 18) 72.18% [11111001000100100000001011111] [100, 18, 1, 1, '4.70', 'log2', 73, '3.57'] *  
 19) 72.18% [11111001000100100000001011111] [100, 18, 1, 1, '4.70', 'log2', 73, '3.57']    
 20) 72.16% [11101001000100100000001001001] [100, 18, 1, 1, '4.70', 'log2', 73, '3.57']    
 21) 72.16% [11101001000100100000001001001] [100, 18, 1, 1, '4.70', 'log2', 78, '3.57']    
 22) 72.16% [01100011010101101000000001010] [100, 16, 2, 1, '4.25', 'log2', 45, '2.90'] *  
 23) 72.14% [11101111110111111010100001001] [100, 18, 2, 2, '4.29', 'log2', 79, '2.24'] *  
 24) 72.12% [11101000110101110000110011001] [100, 18, 1, 1, '4.70', 'log2', 73, '3.57']    
 25) 72.11% [11111001000101111010100001010] [100, 14, 2, 1, '4.25', 'sqrt', 73, '3.57']    
 26) 72.08% [11101001000100100000001001110] [100, 14, 2, 1, '4.70', 'log2', 78, '3.57'] *  
 27) 72.08% [11101001010111101000001001110] [100, 18, 1, 1, '4.70', 'log2', 73, '3.57'] *  
 28) 72.07% [11100011010101100000001010010] [100, 16, 8, 1, '4.42', 'sqrt', 78, '0.70'] *  
 29) 72.07% [11101111110111101000001011100] [100, 18, 1, 1, '4.70', 'log2', 73, '3.57'] *  
 30) 72.07% [11101111110111101000001011101] [100, 18, 2, 2, '4.29', 'log2', 79, '2.24'] *  
 31) 72.07% [11111001000100111010100001011] [100, 18, 1, 1, '4.70', 'log2', 73, '3.57']    
 32) 72.04% [11111110110100111010100001010] [100, 18, 6, 3, '4.43', 'log2', 79, '2.24']    
 33) 72.04% [11101000110101111010011001110] [100, 14, 2, 1, '4.25', 'log2', 45, '0.70'] *  
 34) 72.01% [11001111000100100000001011010] [100, 14, 2, 1, '4.25', 'sqrt', 45, '2.90'] *  
 35) 71.99% [01101111110111101001010010110] [100, 14, 2, 1, '4.25', 'log2', 45, '0.70']    
 36) 71.96% [01100011010101100000000001010] [100, 16, 2, 1, '4.25', 'log2', 45, '2.90'] *  
 37) 71.95% [11001011000100100000001011010] [100, 16, 2, 1, '4.25', 'log2', 45, '2.90'] *  
 38) 71.95% [11100011010100111010100001010] [100, 14, 2, 1, '4.42', 'sqrt', 78, '0.70'] *  
 39) 71.92% [01101111110111101001010010110] [100, 16, 8, 1, '4.25', 'log2', 45, '0.70'] *  
 40) 71.91% [11001011010101100000001001010] [100, 14, 2, 1, '4.25', 'sqrt', 45, '2.90'] *  
 41) 71.90% [11101000110101110001110010001] [100, 18, 1, 1, '4.25', 'log2', 45, '0.70'] *  
 42) 71.90% [01100011010101111010100001010] [100, 16, 2, 1, '4.25', 'log2', 45, '2.90']    
 43) 71.89% [11100011010100111010100001010] [100, 16, 8, 1, '4.42', 'sqrt', 78, '0.70']    
 44) 71.89% [11100011010100111010100001010] [100, 16, 8, 1, '4.42', 'sqrt', 78, '0.70'] *  
 45) 71.88% [11011101011111111010100001010] [100, 16, 8, 1, '4.42', 'sqrt', 28, '2.62']    
 46) 71.88% [11011101011111111010100001010] [100, 16, 8, 1, '4.42', 'sqrt', 28, '2.62']    
 47) 71.87% [11101011110111010010100001001] [100, 18, 1, 1, '4.70', 'log2', 79, '2.24'] *  
 48) 71.86% [11011011010101111010100001010] [100, 16, 8, 1, '4.42', 'sqrt', 28, '2.62'] *  
 49) 71.82% [11101000110100100000001010001] [100, 17, 2, 2, '4.29', 'log2', 79, '2.24']    
 50) 71.82% [11001011010101111010100001010] [100, 14, 2, 1, '4.25', 'sqrt', 45, '2.90']    

==================  10 ==================
  1) 72.63% [11101111110111101000000001001] [100, 18, 2, 2, '4.29', 'log2', 79, '2.24']    
  2) 72.58% [11111001000100100000000001000] [100, 15, 2, 1, '4.87', 'sqrt', 86, '2.86']    
  3) 72.48% [11111001000100100000001011010] [100, 18, 1, 1, '4.70', 'log2', 73, '3.57'] *  
  4) 72.44% [11101000110111101000000001001] [100, 18, 1, 1, '4.70', 'log2', 73, '3.57']    
  5) 72.40% [11101110110100111010100001010] [100, 16, 8, 1, '4.42', 'sqrt', 28, '2.62']    
  6) 72.32% [11111011101100100000001011111] [100, 18, 1, 1, '4.70', 'log2', 73, '3.57']    
  7) 72.32% [11111011101100100000001011111] [100, 18, 1, 1, '4.70', 'log2', 73, '3.57'] *  
  8) 72.32% [11111001010100111010100001010] [100, 16, 8, 1, '4.42', 'sqrt', 78, '0.70'] *  
  9) 72.31% [11101111100100100000001011100] [100, 15, 1, 1, '4.70', 'log2', 86, '2.75'] *  
 10) 72.30% [11111001000100100000001011100] [100, 18, 1, 1, '4.70', 'log2', 73, '3.57']    
 11) 72.30% [11111001000100100000001011100] [100, 18, 1, 1, '4.25', 'log2', 73, '3.57']    
 12) 72.30% [11111001000100100000001011100] [100, 18, 1, 1, '4.70', 'log2', 73, '3.57'] *  
 13) 72.29% [11101110110100111010100001010] [100, 14, 2, 1, '4.25', 'log2', 78, '2.62']    
 14) 72.28% [11101110110100100000001011100] [100, 18, 1, 1, '4.70', 'log2', 73, '3.57']    
 15) 72.28% [11101110110100100000001011100] [100, 18, 1, 1, '4.25', 'log2', 73, '3.57'] *  
 16) 72.28% [11101110110100100000001011100] [100, 18, 1, 1, '4.70', 'log2', 73, '3.57'] *  
 17) 72.28% [11111011010100100000001011110] [100, 18, 1, 1, '4.70', 'log2', 45, '3.57']    
 18) 72.28% [11111011010100100000001011110] [100, 18, 1, 1, '4.70', 3, 17, '3.57'] *  
 19) 72.25% [11101110110100100000001001110] [100, 14, 2, 1, '4.25', 'log2', 45, '0.70'] *  
 20) 72.25% [11101000110100100000001011110] [100, 16, 8, 1, '4.42', 'log2', 79, '2.24'] *  
 21) 72.25% [11111001000100100000001011100] [100, 15, 2, 1, '4.87', 'sqrt', 86, '2.86']    
 22) 72.25% [11111001000100100000001011100] [100, 15, 2, 1, '4.87', 'sqrt', 86, '2.86']    
 23) 72.25% [11101110110100100000001011101] [100, 14, 2, 1, '4.42', 'sqrt', 28, '2.62']    
 24) 72.24% [11111011010100100000001011111] [100, 18, 1, 1, '4.70', 'log2', 73, '3.57']    
 25) 72.24% [11111110110100111000100001010] [100, 18, 6, 3, '4.43', 'log2', 79, '2.24']    
 26) 72.24% [11111011010100111010100001011] [100, 18, 1, 1, '4.70', 'log2', 73, '3.57'] *  
 27) 72.23% [11111001000100100000001001110] [100, 14, 2, 1, '4.70', 'log2', 73, '3.57'] *  
 28) 72.22% [11111011010100100000001001110] [100, 18, 1, 1, '4.70', 'log2', 45, '3.57'] *  
 29) 72.21% [11101000110100111010100001001] [100, 17, 2, 2, '4.29', 'log2', 79, '2.24']    
 30) 72.20% [01111001000100100000000001110] [100, 18, 1, 1, '4.70', 'log2', 45, '1.75'] *  
 31) 72.20% [11101100110100111010100001001] [100, 17, 2, 2, '4.29', 'log2', 79, '2.24'] *  
 32) 72.20% [11111001010110100000001011100] [100, 18, 2, 1, '4.87', 'sqrt', 86, '2.86'] *  
 33) 72.19% [11101110110100100000001001001] [100, 18, 2, 1, '4.42', 'sqrt', 28, '2.62'] *  
 34) 72.19% [11101111110110100000001011100] [100, 18, 1, 1, '4.70', 'log2', 86, '2.75']    
 35) 72.18% [11111001000100100000001011111] [100, 18, 1, 1, '4.70', 'log2', 73, '3.57']    
 36) 72.18% [11111001000100100000001011111] [100, 18, 1, 1, '4.70', 'log2', 73, '3.57'] *  
 37) 72.18% [11111001000100100000001011111] [100, 18, 1, 1, '4.70', 'log2', 73, '3.57']    
 38) 72.16% [11101001000100100000001001001] [100, 18, 1, 1, '4.70', 'log2', 73, '3.57']    
 39) 72.16% [11101001000100100000001001001] [100, 18, 1, 1, '4.70', 'log2', 78, '3.57']    
 40) 72.16% [11101001000100100000001001001] [100, 18, 1, 1, '4.70', 'sqrt', 73, '3.57'] *  
 41) 72.16% [11101001000100100000001001001] [100, 18, 1, 1, '4.70', 'log2', 78, '3.57'] *  
 42) 72.16% [01100011010101101000000001010] [100, 16, 2, 1, '4.25', 'log2', 45, '2.90']    
 43) 72.14% [11111011010100101000001011111] [100, 18, 1, 1, '4.70', 'log2', 73, '3.57'] *  
 44) 72.14% [11101111110111111010100001001] [100, 18, 2, 2, '4.29', 'log2', 79, '2.24']    
 45) 72.12% [11101000110101110000110011001] [100, 18, 1, 1, '4.70', 'log2', 73, '3.57']    
 46) 72.11% [11111001000101111010100001010] [100, 14, 2, 1, '4.25', 'sqrt', 73, '3.57']    
 47) 72.11% [11111001000101111010100001010] [100, 14, 2, 1, '4.25', 'log2', 73, '3.57'] *  
 48) 72.11% [11101001000100100000001011101] [100, 14, 1, 1, '4.70', 'log2', 78, '3.57'] *  
 49) 72.10% [11101001000100100000001011111] [100, 18, 1, 1, '4.70', 'log2', 78, '3.57'] *  
 50) 72.08% [11101001000100100000001001110] [100, 14, 2, 1, '4.70', 'log2', 78, '3.57']    

==================  11 ==================
  1) 72.63% [11101111110111101000000001001] [100, 18, 2, 2, '4.29', 'log2', 79, '2.24']    
  2) 72.58% [11111001000100100000000001000] [100, 15, 2, 1, '4.87', 'sqrt', 86, '2.86']    
  3) 72.49% [11111001000100111010100001000] [100, 15, 2, 1, '4.87', 'sqrt', 86, '2.86'] *  
  4) 72.48% [11111001000100100000001011010] [100, 18, 1, 1, '4.70', 'log2', 73, '3.57']    
  5) 72.46% [11111110110100111000100001010] [100, 18, 1, 1, '4.43', 'log2', 79, '2.24'] *  
  6) 72.44% [11101000110111101000000001001] [100, 18, 1, 1, '4.70', 'log2', 73, '3.57']    
  7) 72.42% [11111001000101101000000001010] [100, 16, 2, 1, '4.25', 'log2', 45, '3.57'] *  
  8) 72.40% [11101110110100111010100001010] [100, 16, 8, 1, '4.42', 'sqrt', 28, '2.62']    
  9) 72.38% [11111110110100111010100001010] [100, 14, 2, 1, '4.25', 'log2', 78, '2.62'] *  
 10) 72.37% [11101110110100100000001011001] [100, 18, 1, 1, '4.70', 'log2', 73, '3.57'] *  
 11) 72.33% [11111011010100101000001001001] [100, 18, 1, 1, '4.70', 'log2', 73, '3.57'] *  
 12) 72.32% [11111011101100100000001011111] [100, 18, 1, 1, '4.70', 'log2', 73, '3.57']    
 13) 72.32% [11111011101100100000001011111] [100, 18, 1, 1, '4.70', 'log2', 73, '3.57']    
 14) 72.32% [11101111110111101000001011100] [100, 15, 2, 1, '4.70', 'log2', 73, '3.57'] *  
 15) 72.32% [11111001010100111010100001010] [100, 16, 8, 1, '4.42', 'sqrt', 78, '0.70']    
 16) 72.31% [11101111100100100000001011100] [100, 15, 1, 1, '4.70', 'log2', 86, '2.75']    
 17) 72.31% [11101001100100100000001011110] [100, 14, 2, 1, '4.70', 'log2', 78, '3.57'] *  
 18) 72.30% [11111001000100100000001011100] [100, 18, 1, 1, '4.70', 'log2', 73, '3.57']    
 19) 72.30% [11111001000100100000001011100] [100, 18, 1, 1, '4.25', 'log2', 73, '3.57']    
 20) 72.30% [11111001000100100000001011100] [100, 18, 1, 1, '4.70', 'log2', 73, '3.57']    
 21) 72.30% [11111001000100100000001011100] [100, 18, 1, 1, '4.25', 'log2', 73, '1.75'] *  
 22) 72.30% [11111001000100100000001011100] [100, 18, 1, 1, '4.87', 'sqrt', 86, '2.86'] *  
 23) 72.29% [11101110110100111010100001010] [100, 14, 2, 1, '4.25', 'log2', 78, '2.62']    
 24) 72.28% [11101110110100111010100001001] [100, 17, 2, 2, '4.29', 'log2', 79, '2.24'] *  
 25) 72.28% [11101110110100100000001011100] [100, 18, 1, 1, '4.70', 'log2', 73, '3.57']    
 26) 72.28% [11101110110100100000001011100] [100, 18, 1, 1, '4.25', 'log2', 73, '3.57']    
 27) 72.28% [11101110110100100000001011100] [100, 18, 1, 1, '4.70', 'log2', 73, '3.57']    
 28) 72.28% [11101110110100100000001011100] [100, 18, 1, 1, '4.70', 'log2', 79, '2.24'] *  
 29) 72.28% [11101110110100100000001011100] [100, 18, 1, 1, '4.70', 'log2', 73, '3.57'] *  
 30) 72.28% [11111011010100100000001011110] [100, 18, 1, 1, '4.70', 'log2', 45, '3.57']    
 31) 72.28% [11111011010100100000001011110] [100, 18, 1, 1, '4.70', 3, 17, '3.57']    
 32) 72.28% [11111011010100100000001011110] [100, 18, 1, 1, '4.70', 'log2', 73, '3.57'] *  
 33) 72.26% [11101111110110100000100001100] [100, 18, 1, 1, '4.70', 'log2', 86, '2.75'] *  
 34) 72.26% [11101110110100100000001011110] [100, 14, 2, 1, '4.70', 'log2', 73, '3.57'] *  
 35) 72.25% [11101110110100100000001001110] [100, 14, 2, 1, '4.25', 'log2', 45, '0.70']    
 36) 72.25% [11101110110100100000001001110] [100, 14, 2, 1, '4.25', 'log2', 73, '0.70'] *  
 37) 72.25% [11101000110100100000001011110] [100, 16, 8, 1, '4.42', 'log2', 79, '2.24']    
 38) 72.25% [11101011010100100000001011110] [100, 18, 1, 1, '4.70', 'log2', 45, '3.57'] *  
 39) 72.25% [11111001000100100000001011100] [100, 15, 2, 1, '4.87', 'sqrt', 86, '2.86']    
 40) 72.25% [11111001000100100000001011100] [100, 15, 2, 1, '4.87', 'sqrt', 86, '2.86']    
 41) 72.25% [11101110110100100000001011101] [100, 14, 2, 1, '4.42', 'sqrt', 28, '2.62']    
 42) 72.24% [11111011010100100000001011111] [100, 18, 1, 1, '4.70', 'log2', 73, '3.57']    
 43) 72.24% [11111110110100111000100001010] [100, 18, 6, 3, '4.43', 'log2', 79, '2.24']    
 44) 72.24% [11111011010100111010100001011] [100, 18, 1, 1, '4.70', 'log2', 73, '3.57']    
 45) 72.23% [11111001000100100000001001110] [100, 14, 2, 1, '4.70', 'log2', 73, '3.57']    
 46) 72.23% [11111001000100100000001001100] [100, 15, 2, 1, '4.87', 'sqrt', 86, '2.86'] *  
 47) 72.22% [11111011010100100000001001110] [100, 18, 1, 1, '4.70', 'log2', 45, '3.57']    
 48) 72.22% [11101001000100100000001011001] [100, 18, 1, 1, '4.70', 'log2', 78, '3.57'] *  
 49) 72.21% [11101000110100111010100001001] [100, 17, 2, 2, '4.29', 'log2', 79, '2.24']    
 50) 72.21% [11101000110100100000001001001] [100, 18, 1, 1, '4.25', 'log2', 45, '0.70'] *  

==================  12 ==================
  1) 72.63% [11101111110111101000000001001] [100, 18, 2, 2, '4.29', 'log2', 79, '2.24']    
  2) 72.63% [11101111110111101000000001001] [100, 18, 2, 2, '4.29', 'log2', 79, '2.24'] *  
  3) 72.58% [11111001000100100000000001000] [100, 15, 2, 1, '4.87', 'sqrt', 86, '2.86']    
  4) 72.58% [11111001000100100000000001000] [100, 15, 2, 1, '4.70', 'log2', 78, '2.86'] *  
  5) 72.53% [11101011010111101000000001001] [100, 18, 1, 1, '4.70', 'log2', 73, '3.57'] *  
  6) 72.49% [11111001000100111010100001000] [100, 15, 2, 1, '4.87', 'sqrt', 86, '2.86']    
  7) 72.48% [11111001000100100000001011010] [100, 18, 1, 1, '4.70', 'log2', 73, '3.57']    
  8) 72.47% [11111011101100100000001011010] [100, 18, 1, 1, '4.70', 'log2', 73, '3.57'] *  
  9) 72.46% [11111110110100111000100001010] [100, 18, 2, 1, '4.43', 'log2', 79, '2.24'] *  
 10) 72.46% [11111110110100111000100001010] [100, 18, 1, 1, '4.43', 'log2', 79, '2.24']    
 11) 72.44% [11111010110100100000001011100] [100, 18, 1, 1, '4.70', 'log2', 73, '3.57'] *  
 12) 72.44% [11101000110111101000000001001] [100, 18, 1, 1, '4.70', 'log2', 73, '3.57']    
 13) 72.44% [11101000110111101000000001001] [100, 18, 1, 1, '4.25', 'log2', 73, '3.57'] *  
 14) 72.43% [11101110110100100000001001010] [100, 16, 8, 1, '4.42', 'sqrt', 28, '2.62'] *  
 15) 72.42% [11111001000101101000000001010] [100, 16, 2, 1, '4.25', 'log2', 45, '3.57']    
 16) 72.42% [11111011010100100000000001110] [100, 18, 1, 1, '4.70', 'log2', 45, '1.75'] *  
 17) 72.42% [11111111110110100000001011100] [100, 18, 1, 1, '4.25', 'log2', 73, '3.57'] *  
 18) 72.41% [11101110110100100000001001100] [100, 15, 1, 1, '4.70', 'log2', 86, '2.75'] *  
 19) 72.41% [11111011010100100000001001001] [100, 18, 1, 1, '4.70', 'log2', 73, '3.57'] *  
 20) 72.40% [11101110110100111010100001010] [100, 16, 8, 1, '4.42', 'sqrt', 28, '2.62']    
 21) 72.39% [11111011010100100000001011100] [100, 15, 2, 1, '4.70', 'log2', 45, '3.57'] *  
 22) 72.38% [11111110110100111010100001010] [100, 14, 2, 1, '4.25', 'log2', 78, '2.62']    
 23) 72.37% [11101110110100100000001011001] [100, 18, 1, 1, '4.70', 'log2', 73, '3.57']    
 24) 72.36% [11101110110100100000100001001] [100, 17, 2, 2, '4.29', 'log2', 79, '2.24'] *  
 25) 72.34% [11101111110100100000001001001] [100, 18, 2, 2, '4.29', 'log2', 79, '2.24'] *  
 26) 72.33% [11111011010100101000001001001] [100, 18, 1, 1, '4.70', 'log2', 73, '3.57']    
 27) 72.32% [11111011101100100000001011111] [100, 18, 1, 1, '4.70', 'log2', 73, '3.57']    
 28) 72.32% [11111011101100100000001011111] [100, 18, 1, 1, '4.70', 'log2', 73, '3.57']    
 29) 72.32% [11101111110111101000001011100] [100, 15, 2, 1, '4.70', 'log2', 73, '3.57']    
 30) 72.32% [11101111110111101000001011100] [100, 15, 2, 1, '4.70', 'log2', 73, '3.57'] *  
 31) 72.32% [11111001010100111010100001010] [100, 16, 8, 1, '4.42', 'sqrt', 78, '0.70']    
 32) 72.31% [11101111100100100000001011100] [100, 15, 1, 1, '4.70', 'log2', 86, '2.75']    
 33) 72.31% [11101001100100100000001011110] [100, 14, 2, 1, '4.70', 'log2', 78, '3.57']    
 34) 72.31% [11101001100100100000001011110] [100, 14, 2, 1, '4.87', 'sqrt', 86, '3.57'] *  
 35) 72.30% [11101110110100100010100001010] [100, 16, 8, 1, '4.42', 'sqrt', 28, '2.62'] *  
 36) 72.30% [11111001000100100000001011100] [100, 18, 1, 1, '4.70', 'log2', 73, '3.57']    
 37) 72.30% [11111001000100100000001011100] [100, 18, 1, 1, '4.25', 'log2', 73, '3.57']    
 38) 72.30% [11111001000100100000001011100] [100, 18, 1, 1, '4.70', 'log2', 73, '3.57']    
 39) 72.30% [11111001000100100000001011100] [100, 18, 1, 1, '4.25', 'log2', 73, '1.75']    
 40) 72.30% [11111001000100100000001011100] [100, 18, 1, 1, '4.87', 'sqrt', 86, '2.86']    
 41) 72.30% [11111001000100100000001011100] [100, 18, 1, 1, '4.70', 'log2', 73, '3.57'] *  
 42) 72.30% [11111001000100100000001011100] [100, 18, 1, 1, '4.25', 'log2', 73, '3.57'] *  
 43) 72.30% [11111001000100100000001011100] [100, 18, 1, 1, '4.70', 'log2', 73, '1.75'] *  
 44) 72.30% [11111001000100100000001011100] [100, 18, 1, 1, '4.70', 'log2', 73, '3.57'] *  
 45) 72.30% [11111001000100100000001011100] [100, 18, 1, 1, '4.25', 'log2', 79, '2.24'] *  
 46) 72.30% [11111001000100100000001011100] [100, 18, 1, 1, '4.25', 'log2', 73, '3.57'] *  
 47) 72.29% [11101110110100111010100001010] [100, 14, 2, 1, '4.25', 'log2', 78, '2.62']    
 48) 72.29% [11111011110100100000001011111] [100, 18, 1, 1, '4.70', 'log2', 73, '3.57'] *  
 49) 72.29% [11111001000100100000001011001] [100, 18, 1, 1, '4.70', 'log2', 78, '3.57'] *  
 50) 72.29% [11111001000100100000001011001] [100, 18, 1, 1, '4.70', 'log2', 73, '3.57'] *  

==================  13 ==================
  1) 72.94% [11111110110100100000000001000] [100, 15, 2, 1, '4.87', 'sqrt', 86, '2.86'] *  
  2) 72.73% [11111011110111101000000001001] [100, 18, 2, 2, '4.29', 'log2', 79, '2.24'] *  
  3) 72.65% [11101111110111101000000001001] [100, 18, 1, 1, '4.70', 'log2', 79, '2.24'] *  
  4) 72.64% [11101110110100101000000001001] [100, 18, 2, 2, '4.29', 'log2', 79, '2.24'] *  
  5) 72.64% [11111011010100100000001011000] [100, 15, 2, 1, '4.87', 'sqrt', 86, '2.86'] *  
  6) 72.63% [11101111110111101000000001001] [100, 18, 2, 2, '4.29', 'log2', 79, '2.24']    
  7) 72.63% [11101111110111101000000001001] [100, 18, 2, 2, '4.29', 'log2', 79, '2.24']    
  8) 72.60% [11111110110100111010000001010] [100, 14, 2, 1, '4.25', 'log2', 78, '2.62'] *  
  9) 72.58% [11111001000100100000000001000] [100, 15, 2, 1, '4.70', 'log2', 78, '2.86'] *  
 10) 72.58% [11111001000100100000000001000] [100, 15, 2, 1, '4.87', 'sqrt', 86, '2.86']    
 11) 72.58% [11111001000100100000000001000] [100, 15, 2, 1, '4.70', 'log2', 78, '2.86']    
 12) 72.53% [11101011010111101000000001001] [100, 18, 1, 1, '4.70', 'log2', 73, '3.57']    
 13) 72.53% [11101011010111101000000001001] [100, 18, 1, 1, '4.70', 'log2', 73, '3.57'] *  
 14) 72.49% [11111001000100111010100001000] [100, 15, 2, 1, '4.87', 'sqrt', 86, '2.86']    
 15) 72.48% [11111001000100100000001011010] [100, 18, 1, 1, '4.70', 'log2', 73, '3.57']    
 16) 72.48% [11101111010111101000000001001] [100, 18, 2, 2, '4.29', 'log2', 79, '2.24'] *  
 17) 72.48% [11101011110111101000000001001] [100, 18, 1, 1, '4.70', 'log2', 73, '3.57'] *  
 18) 72.47% [11111011101100100000001011010] [100, 18, 1, 1, '4.70', 'log2', 73, '3.57']    
 19) 72.46% [11111110110100111000100001010] [100, 18, 2, 1, '4.43', 'log2', 79, '2.24']    
 20) 72.46% [11111110110100111000100001010] [100, 18, 1, 1, '4.43', 'log2', 79, '2.24']    
 21) 72.44% [11111010110100100000001011100] [100, 18, 1, 1, '4.70', 'log2', 73, '3.57']    
 22) 72.44% [11111010110100100000001011100] [100, 18, 1, 1, '4.70', 'log2', 73, '3.57'] *  
 23) 72.44% [11101000110111101000000001001] [100, 18, 1, 1, '4.70', 'log2', 73, '3.57']    
 24) 72.44% [11101000110111101000000001001] [100, 18, 1, 1, '4.25', 'log2', 73, '3.57']    
 25) 72.43% [11101110110100100000001001010] [100, 16, 8, 1, '4.42', 'sqrt', 28, '2.62']    
 26) 72.43% [11101110110100100000001001010] [100, 16, 8, 1, '4.42', 'sqrt', 28, '3.57'] *  
 27) 72.42% [11111001000101101000000001010] [100, 16, 2, 1, '4.25', 'log2', 45, '3.57']    
 28) 72.42% [11111011010100100000000001110] [100, 18, 1, 1, '4.70', 'log2', 45, '1.75']    
 29) 72.42% [11111111110110100000001011100] [100, 18, 1, 1, '4.25', 'log2', 73, '3.57']    
 30) 72.42% [11111111110110100000001011100] [100, 18, 1, 1, '4.25', 'log2', 73, '3.57'] *  
 31) 72.41% [11101110110100100000001001100] [100, 15, 1, 1, '4.70', 'log2', 86, '2.75']    
 32) 72.41% [11111011010100100000001001001] [100, 18, 1, 1, '4.70', 'log2', 73, '3.57']    
 33) 72.41% [11111011010100100000001001001] [100, 18, 1, 1, '4.70', 'log2', 73, '3.57'] *  
 34) 72.40% [11101110110100111010100001010] [100, 16, 8, 1, '4.42', 'sqrt', 28, '2.62']    
 35) 72.40% [11101110110100111010100001010] [100, 16, 8, 1, '4.25', 'log2', 28, '2.62'] *  
 36) 72.40% [11101110110100111010100001010] [100, 16, 8, 1, '4.42', 'sqrt', 28, '3.57'] *  
 37) 72.39% [11111001000100100000001001000] [100, 15, 2, 1, '4.87', 'sqrt', 86, '2.86'] *  
 38) 72.39% [11111011010100100000001011100] [100, 15, 2, 1, '4.70', 'log2', 45, '3.57']    
 39) 72.38% [11111110110100111010100001010] [100, 14, 2, 1, '4.25', 'log2', 78, '2.62']    
 40) 72.37% [11111001000100100000000011100] [100, 18, 1, 1, '4.70', 'log2', 73, '3.57'] *  
 41) 72.37% [11101110110100100000001011001] [100, 18, 1, 1, '4.70', 'log2', 73, '3.57']    
 42) 72.37% [11101110110100100000001011001] [100, 18, 1, 1, '4.70', 'log2', 73, '2.62'] *  
 43) 72.36% [11101110110100100000100001001] [100, 17, 2, 2, '4.29', 'log2', 79, '2.24']    
 44) 72.36% [11101110110100100000100001001] [100, 17, 2, 2, '4.29', 'log2', 79, '2.24'] *  
 45) 72.35% [11111110110100100010100001010] [100, 18, 1, 1, '4.43', 'log2', 79, '2.24'] *  
 46) 72.34% [11101111110100100000001001001] [100, 18, 2, 2, '4.29', 'log2', 79, '2.24']    
 47) 72.34% [11101111110100100000001001001] [100, 18, 2, 2, '4.29', 'log2', 79, '2.24'] *  
 48) 72.33% [11111011010100100000001011100] [100, 18, 1, 1, '4.70', 'log2', 73, '3.57'] *  
 49) 72.33% [11111011010100101000001001001] [100, 18, 1, 1, '4.70', 'log2', 73, '3.57']    
 50) 72.32% [11101001110100101000001011110] [100, 14, 2, 1, '4.70', 'log2', 78, '3.57'] *  

==================  14 ==================
  1) 72.94% [11111110110100100000000001000] [100, 15, 2, 1, '4.87', 'sqrt', 86, '2.86']    
  2) 72.84% [11111011101100100000000001000] [100, 15, 2, 1, '4.70', 'log2', 73, '3.57'] *  
  3) 72.73% [11111011110111101000000001001] [100, 18, 2, 2, '4.29', 'log2', 79, '2.24']    
  4) 72.72% [11101110110100100000001011000] [100, 15, 2, 1, '4.87', 'sqrt', 86, '2.86'] *  
  5) 72.70% [11111010110100100000000001010] [100, 18, 1, 1, '4.43', 'log2', 79, '2.24'] *  
  6) 72.65% [11101111110111101000000001001] [100, 18, 1, 1, '4.70', 'log2', 79, '2.24']    
  7) 72.64% [11101110110100101000000001001] [100, 18, 2, 2, '4.29', 'log2', 79, '2.24']    
  8) 72.64% [11101110110100101000000001001] [100, 18, 2, 2, '4.29', 'log2', 73, '2.24'] *  
  9) 72.64% [11111011010100100000001011000] [100, 15, 2, 1, '4.87', 'sqrt', 86, '2.86']    
 10) 72.64% [11111011010100100000001011000] [100, 15, 2, 1, '4.87', 'sqrt', 73, '3.57'] *  
 11) 72.63% [11101111110111101000000001001] [100, 18, 2, 2, '4.29', 'log2', 79, '2.24']    
 12) 72.63% [11101111110111101000000001001] [100, 18, 2, 2, '4.29', 'log2', 79, '2.24']    
 13) 72.61% [11111011010100100000000001001] [100, 18, 2, 2, '4.29', 'log2', 73, '3.57'] *  
 14) 72.60% [11101110110100100000000001010] [100, 16, 8, 1, '4.42', 'sqrt', 28, '2.62'] *  
 15) 72.60% [11101110110100100000000001010] [100, 16, 8, 1, '4.42', 'sqrt', 28, '3.57'] *  
 16) 72.60% [11111110110100111010000001010] [100, 14, 2, 1, '4.25', 'log2', 78, '2.62']    
 17) 72.58% [11111001000100100000000001000] [100, 15, 2, 1, '4.70', 'log2', 78, '2.86']    
 18) 72.58% [11111001000100100000000001000] [100, 15, 2, 1, '4.87', 'sqrt', 86, '2.86']    
 19) 72.58% [11111001000100100000000001000] [100, 15, 2, 1, '4.70', 'log2', 78, '2.86']    
 20) 72.58% [11111001000100100000000001000] [100, 15, 2, 1, '4.70', 'log2', 78, '0.70'] *  
 21) 72.56% [11111011110100100000001001010] [100, 16, 8, 1, '4.70', 'log2', 73, '3.57'] *  
 22) 72.54% [11111010110100100000001001010] [100, 16, 8, 1, '4.70', 'log2', 73, '3.57'] *  
 23) 72.53% [11101011010111101000000001001] [100, 18, 1, 1, '4.70', 'log2', 73, '3.57']    
 24) 72.53% [11101011010111101000000001001] [100, 18, 1, 1, '4.70', 'log2', 73, '3.57']    
 25) 72.53% [11101011010111101000000001001] [100, 18, 1, 1, '4.70', 'log2', 73, '3.57'] *  
 26) 72.52% [11111110110100111000001011010] [100, 14, 2, 1, '4.25', 'log2', 78, '2.62'] *  
 27) 72.50% [11111011010100101000001001010] [100, 16, 1, 1, '4.70', 'log2', 73, '3.57'] *  
 28) 72.50% [11111111000100100000000011010] [100, 14, 2, 1, '4.25', 'log2', 78, '2.62'] *  
 29) 72.49% [11111001000100111010100001000] [100, 15, 2, 1, '4.87', 'sqrt', 86, '2.86']    
 30) 72.48% [11111000110100111010000001100] [100, 18, 1, 1, '4.70', 'log2', 73, '3.57'] *  
 31) 72.48% [11111001000100100000001011010] [100, 18, 1, 1, '4.70', 'log2', 73, '3.57']    
 32) 72.48% [11111001000100100000001011010] [100, 18, 1, 1, '4.70', 'log2', 78, '2.86'] *  
 33) 72.48% [11111001000100100000001011010] [100, 18, 1, 1, '4.25', 'log2', 73, '3.57'] *  
 34) 72.48% [11101111010111101000000001001] [100, 18, 2, 2, '4.29', 'log2', 79, '2.24']    
 35) 72.48% [11111010110100100000001001001] [100, 18, 1, 1, '4.70', 'log2', 73, '3.57'] *  
 36) 72.48% [11101011110111101000000001001] [100, 18, 1, 1, '4.70', 'log2', 73, '3.57']    
 37) 72.48% [11111011010100100000001001010] [100, 16, 2, 1, '4.25', 'log2', 45, '2.86'] *  
 38) 72.48% [11111011010100100000001001010] [100, 16, 8, 1, '4.42', 'sqrt', 28, '2.62'] *  
 39) 72.47% [11111011101100100000001011010] [100, 18, 1, 1, '4.70', 'log2', 73, '3.57']    
 40) 72.47% [11111011101100100000001011010] [100, 18, 8, 1, '4.42', 'log2', 73, '3.57'] *  
 41) 72.46% [11111110110100111000100001010] [100, 18, 2, 1, '4.43', 'log2', 79, '2.24']    
 42) 72.46% [11111110110100111000100001010] [100, 18, 1, 1, '4.43', 'log2', 79, '2.24']    
 43) 72.44% [11111010110100100000001011100] [100, 18, 1, 1, '4.70', 'log2', 73, '3.57']    
 44) 72.44% [11111010110100100000001011100] [100, 18, 1, 1, '4.70', 'log2', 73, '3.57']    
 45) 72.44% [11111010110100100000001011100] [100, 18, 1, 1, '4.70', 'log2', 73, '3.57'] *  
 46) 72.44% [11111010110100100000001011100] [100, 18, 1, 1, '4.25', 'log2', 73, '3.57'] *  
 47) 72.44% [11101000110111101000000001001] [100, 18, 1, 1, '4.70', 'log2', 73, '3.57']    
 48) 72.44% [11101000110111101000000001001] [100, 18, 1, 1, '4.25', 'log2', 73, '3.57']    
 49) 72.44% [11101000110111101000000001001] [100, 18, 1, 1, '4.25', 'sqrt', 73, '3.57'] *  
 50) 72.44% [11101000110111101000000001001] [100, 18, 1, 1, '4.70', 'log2', 73, '3.57'] *  

==================  15 ==================
  1) 73.09% [11101111110111101000000001000] [100, 15, 2, 1, '4.87', 'sqrt', 86, '2.86'] *  
  2) 72.94% [11111110110100100000000001000] [100, 15, 2, 1, '4.87', 'sqrt', 86, '2.86']    
  3) 72.87% [11111011110111101000000001010] [100, 16, 8, 1, '4.29', 'log2', 79, '2.24'] *  
  4) 72.84% [11111011101100100000000001000] [100, 15, 2, 1, '4.70', 'log2', 73, '3.57']    
  5) 72.77% [11111110110100111000000001010] [100, 18, 2, 1, '4.43', 'log2', 79, '2.24'] *  
  6) 72.73% [11111011110111101000000001001] [100, 18, 2, 2, '4.29', 'log2', 79, '2.24']    
  7) 72.72% [11101110110100100000001011000] [100, 15, 2, 1, '4.87', 'sqrt', 86, '2.86']    
  8) 72.72% [11101000110111100000000001010] [100, 16, 8, 1, '4.25', 'sqrt', 73, '3.57'] *  
  9) 72.72% [11111010110100100000000011010] [100, 14, 2, 1, '4.25', 'log2', 78, '2.62'] *  
 10) 72.71% [11101111110111101000000001010] [100, 18, 2, 2, '4.29', 'log2', 79, '2.24'] *  
 11) 72.71% [11111010110100100000000001001] [100, 18, 1, 1, '4.70', 'log2', 73, '3.57'] *  
 12) 72.70% [11111010110100100000000001010] [100, 18, 1, 1, '4.43', 'log2', 79, '2.24']    
 13) 72.68% [11111110110100100000000001001] [100, 18, 2, 2, '4.29', 'log2', 79, '2.24'] *  
 14) 72.66% [11101000110110101000000001001] [100, 18, 1, 1, '4.70', 'log2', 73, '3.57'] *  
 15) 72.65% [11101111110111101000000001001] [100, 18, 1, 1, '4.70', 'log2', 79, '2.24']    
 16) 72.65% [11101111110111101000000001001] [100, 18, 1, 1, '4.70', 'sqrt', 28, '2.24'] *  
 17) 72.64% [11101111110110100000001001010] [100, 16, 8, 1, '4.42', 'sqrt', 28, '3.57'] *  
 18) 72.64% [11111110110100100000000011100] [100, 18, 1, 1, '4.25', 'log2', 73, '3.57'] *  
 19) 72.64% [11101110110100101000000001001] [100, 18, 2, 2, '4.29', 'log2', 79, '2.24']    
 20) 72.64% [11101110110100101000000001001] [100, 18, 2, 2, '4.29', 'log2', 73, '2.24']    
 21) 72.64% [11101110110100101000000001001] [100, 18, 2, 2, '4.43', 'log2', 73, '2.24'] *  
 22) 72.64% [11111011010100100000001011000] [100, 15, 2, 1, '4.87', 'sqrt', 86, '2.86']    
 23) 72.64% [11111011010100100000001011000] [100, 15, 2, 1, '4.87', 'sqrt', 73, '3.57']    
 24) 72.63% [11111000110111101000000001001] [100, 18, 1, 1, '4.25', 'sqrt', 73, '3.57'] *  
 25) 72.63% [11101011010100100000001011000] [100, 15, 2, 1, '4.87', 'sqrt', 73, '3.57'] *  
 26) 72.63% [11101111110111101000000001001] [100, 18, 2, 2, '4.29', 'log2', 79, '2.24']    
 27) 72.63% [11101111110111101000000001001] [100, 18, 2, 2, '4.29', 'log2', 79, '2.24']    
 28) 72.61% [11111011010100100000000001001] [100, 18, 2, 2, '4.29', 'log2', 73, '3.57']    
 29) 72.60% [11101110110100100000000001010] [100, 16, 8, 1, '4.42', 'sqrt', 28, '2.62']    
 30) 72.60% [11101110110100100000000001010] [100, 16, 8, 1, '4.42', 'sqrt', 28, '3.57']    
 31) 72.60% [11101110110100100000000001010] [100, 16, 2, 1, '4.25', 'log2', 28, '3.57'] *  
 32) 72.60% [11101110110100100000000001010] [100, 16, 8, 1, '4.42', 'log2', 79, '2.62'] *  
 33) 72.60% [11111110110100111010000001010] [100, 14, 2, 1, '4.25', 'log2', 78, '2.62']    
 34) 72.59% [11111011010100100000000001001] [100, 18, 1, 1, '4.70', 'log2', 73, '3.57'] *  
 35) 72.58% [11111001000100100000000001000] [100, 15, 2, 1, '4.70', 'log2', 78, '2.86']    
 36) 72.58% [11111001000100100000000001000] [100, 15, 2, 1, '4.87', 'sqrt', 86, '2.86']    
 37) 72.58% [11111001000100100000000001000] [100, 15, 2, 1, '4.70', 'log2', 78, '2.86']    
 38) 72.58% [11111001000100100000000001000] [100, 15, 2, 1, '4.70', 'log2', 78, '0.70']    
 39) 72.58% [11111111110100100000001011010] [100, 18, 1, 1, '4.70', 'log2', 73, '3.57'] *  
 40) 72.56% [11111011110100100000001001010] [100, 16, 8, 1, '4.70', 'log2', 73, '3.57']    
 41) 72.54% [11111010110100100000001001010] [100, 16, 8, 1, '4.70', 'log2', 73, '3.57']    
 42) 72.53% [11101111110111101000000011010] [100, 18, 2, 2, '4.29', 'log2', 79, '2.24'] *  
 43) 72.53% [11111011010100100000001001010] [100, 18, 1, 1, '4.70', 'log2', 73, '3.57'] *  
 44) 72.53% [11101011010111101000000001001] [100, 18, 1, 1, '4.70', 'log2', 73, '3.57']    
 45) 72.53% [11101011010111101000000001001] [100, 18, 1, 1, '4.70', 'log2', 73, '3.57']    
 46) 72.53% [11101011010111101000000001001] [100, 18, 1, 1, '4.70', 'log2', 73, '3.57']    
 47) 72.52% [11111011101100101000000001001] [100, 18, 1, 1, '4.70', 'log2', 73, '3.57'] *  
 48) 72.52% [11111110110100111000001011010] [100, 14, 2, 1, '4.25', 'log2', 78, '2.62']    
 49) 72.50% [11111011010100101000001001010] [100, 16, 1, 1, '4.70', 'log2', 73, '3.57']    
 50) 72.50% [11111111000100100000000011010] [100, 14, 2, 1, '4.25', 'log2', 78, '2.62']    

==================  16 ==================
  1) 73.09% [11101111110111101000000001000] [100, 15, 2, 1, '4.87', 'sqrt', 86, '2.86']    
  2) 72.94% [11111110110100100000000001000] [100, 15, 2, 1, '4.87', 'sqrt', 86, '2.86']    
  3) 72.94% [11111110110100100000000001000] [100, 15, 2, 1, '4.87', 'sqrt', 78, '2.86'] *  
  4) 72.91% [11101011110111101000000001000] [100, 15, 2, 1, '4.70', 'log2', 73, '3.57'] *  
  5) 72.87% [11111011110111101000000001010] [100, 16, 8, 1, '4.29', 'log2', 79, '2.24']    
  6) 72.87% [11111011110111101000000001010] [100, 16, 8, 1, '4.42', 'log2', 79, '2.62'] *  
  7) 72.87% [11111011110111101000000001010] [100, 16, 8, 1, '4.29', 'log2', 78, '2.62'] *  
  8) 72.86% [11101110110100100000000001000] [100, 16, 8, 1, '4.42', 'log2', 79, '2.62'] *  
  9) 72.86% [11101011010111101000000001000] [100, 18, 1, 1, '4.70', 'log2', 73, '3.57'] *  
 10) 72.84% [11111011101100100000000001000] [100, 15, 2, 1, '4.70', 'log2', 73, '3.57']    
 11) 72.83% [11101111110111101000000001000] [100, 18, 2, 2, '4.29', 'log2', 79, '2.24'] *  
 12) 72.80% [11101111110111101000000001010] [100, 16, 8, 1, '4.29', 'log2', 79, '2.24'] *  
 13) 72.77% [11111110110100100000000001010] [100, 16, 8, 1, '4.70', 'log2', 73, '3.57'] *  
 14) 72.77% [11111110110100111000000001010] [100, 18, 2, 1, '4.43', 'log2', 79, '2.24']    
 15) 72.77% [11111110110100111000000001010] [100, 18, 2, 1, '4.43', 'log2', 79, '2.24'] *  
 16) 72.75% [11111010110100100000001011000] [100, 15, 2, 1, '4.87', 'log2', 73, '3.57'] *  
 17) 72.73% [11111011110111101000000001001] [100, 18, 2, 2, '4.29', 'log2', 79, '2.24']    
 18) 72.73% [11111011110111101000000001001] [100, 18, 2, 2, '4.87', 'log2', 79, '2.24'] *  
 19) 72.72% [11101110110100100000001011000] [100, 15, 2, 1, '4.87', 'sqrt', 86, '2.86']    
 20) 72.72% [11101000110111100000000001010] [100, 16, 8, 1, '4.25', 'sqrt', 73, '3.57']    
 21) 72.72% [11111010110100100000000011010] [100, 14, 2, 1, '4.25', 'log2', 78, '2.62']    
 22) 72.72% [11111000110110101000000001001] [100, 18, 1, 1, '4.70', 'log2', 73, '3.57'] *  
 23) 72.71% [11111011101100100000000001010] [100, 16, 8, 1, '4.25', 'sqrt', 73, '3.57'] *  
 24) 72.71% [11101111110111101000000001010] [100, 18, 2, 2, '4.29', 'log2', 79, '2.24']    
 25) 72.71% [11111010110100100000000001001] [100, 18, 1, 1, '4.70', 'log2', 73, '3.57']    
 26) 72.71% [11111010110100100000000001001] [100, 18, 1, 1, '4.70', 'log2', 73, '3.57'] *  
 27) 72.70% [11111010110100100000000001010] [100, 18, 1, 1, '4.43', 'log2', 79, '2.24']    
 28) 72.70% [11101000110111100000000001000] [100, 15, 2, 1, '4.70', 'log2', 73, '3.57'] *  
 29) 72.68% [11111110110100100000000001001] [100, 18, 2, 2, '4.29', 'log2', 79, '2.24']    
 30) 72.68% [11111110110100100000000001001] [100, 18, 2, 2, '4.29', 'log2', 73, '3.57'] *  
 31) 72.68% [11111011110111101000000001001] [100, 18, 1, 1, '4.70', 'log2', 79, '2.24'] *  
 32) 72.68% [11101110110100101000000001010] [100, 18, 2, 2, '4.43', 'log2', 73, '2.24'] *  
 33) 72.68% [11111011010100100000001011000] [100, 18, 1, 1, '4.70', 'log2', 73, '3.57'] *  
 34) 72.67% [11111010110100100000001001000] [100, 16, 8, 1, '4.70', 'log2', 73, '3.57'] *  
 35) 72.66% [11101000110110101000000001001] [100, 18, 1, 1, '4.70', 'log2', 73, '3.57']    
 36) 72.65% [11111011101100100000000001001] [100, 18, 1, 1, '4.70', 'log2', 73, '3.57'] *  
 37) 72.65% [11101111110111101000000001001] [100, 18, 1, 1, '4.70', 'log2', 79, '2.24']    
 38) 72.65% [11101111110111101000000001001] [100, 18, 1, 1, '4.70', 'sqrt', 28, '2.24']    
 39) 72.65% [11101111110111101000000001001] [100, 18, 2, 1, '4.29', 'log2', 79, '2.24'] *  
 40) 72.64% [11101111110110100000001001010] [100, 16, 8, 1, '4.42', 'sqrt', 28, '3.57']    
 41) 72.64% [11101111110110100000001001010] [100, 16, 8, 1, '4.42', 'sqrt', 28, '3.57'] *  
 42) 72.64% [11111110110100100000000011100] [100, 18, 1, 1, '4.25', 'log2', 73, '3.57']    
 43) 72.64% [11111110110100100000000011100] [100, 18, 1, 1, '4.25', 'log2', 78, '2.86'] *  
 44) 72.64% [11101110110100101000000001001] [100, 18, 2, 2, '4.43', 'log2', 73, '2.24'] *  
 45) 72.64% [11101110110100101000000001001] [100, 18, 2, 2, '4.29', 'log2', 79, '2.24']    
 46) 72.64% [11101110110100101000000001001] [100, 18, 2, 2, '4.29', 'log2', 73, '2.24']    
 47) 72.64% [11101110110100101000000001001] [100, 18, 2, 2, '4.43', 'log2', 73, '2.24']    
 48) 72.64% [11111011010100100000001011000] [100, 15, 2, 1, '4.87', 'sqrt', 86, '2.86']    
 49) 72.64% [11111011010100100000001011000] [100, 15, 2, 1, '4.87', 'sqrt', 73, '3.57']    
 50) 72.64% [11111011010100100000001011000] [100, 15, 2, 1, '4.29', 'sqrt', 73, '3.57'] *  

==================  17 ==================
  1) 73.09% [11101111110111101000000001000] [100, 15, 2, 1, '4.87', 'sqrt', 86, '2.86']    
  2) 73.09% [11101111110111101000000001000] [100, 15, 2, 1, '4.87', 'sqrt', 86, '2.86'] *  
  3) 73.02% [11111011110111101000000001000] [100, 15, 2, 1, '4.29', 'log2', 79, '2.24'] *  
  4) 72.98% [11101111110110101000000001000] [100, 18, 2, 2, '4.29', 'log2', 79, '2.24'] *  
  5) 72.94% [11111110110100100000000001000] [100, 15, 2, 1, '4.87', 'sqrt', 86, '2.86']    
  6) 72.94% [11111110110100100000000001000] [100, 15, 2, 1, '4.87', 'sqrt', 78, '2.86']    
  7) 72.94% [11111110110100100000000001000] [100, 15, 2, 1, '4.87', 'sqrt', 86, '2.86'] *  
  8) 72.91% [11101011110111101000000001000] [100, 15, 2, 1, '4.70', 'log2', 73, '3.57']    
  9) 72.91% [11101011110111101000000001000] [100, 15, 2, 1, '4.70', 'log2', 73, '2.24'] *  
 10) 72.91% [11101011110111101000000001000] [100, 15, 2, 1, '4.70', 'log2', 73, '3.57'] *  
 11) 72.87% [11111011110111101000000001010] [100, 16, 8, 1, '4.29', 'log2', 79, '2.24']    
 12) 72.87% [11111011110111101000000001010] [100, 16, 8, 1, '4.42', 'log2', 79, '2.62']    
 13) 72.87% [11111011110111101000000001010] [100, 16, 8, 1, '4.29', 'log2', 78, '2.62']    
 14) 72.87% [11111011110111101000000001010] [100, 16, 2, 1, '4.29', 'log2', 79, '2.24'] *  
 15) 72.86% [11101110110100100000000001000] [100, 16, 8, 1, '4.42', 'log2', 79, '2.62']    
 16) 72.86% [11101110110100100000000001000] [100, 16, 8, 1, '4.42', 'log2', 79, '2.62'] *  
 17) 72.86% [11101110110100100000000001000] [100, 16, 8, 1, '4.42', 'log2', 79, '2.62'] *  
 18) 72.86% [11101011010111101000000001000] [100, 18, 1, 1, '4.70', 'log2', 73, '3.57']    
 19) 72.86% [11101110110100100000000001000] [100, 15, 2, 1, '4.42', 'log2', 79, '2.62'] *  
 20) 72.84% [11111000110111101000000001000] [100, 15, 2, 1, '4.87', 'sqrt', 73, '3.57'] *  
 21) 72.84% [11111011101100100000000001000] [100, 15, 2, 1, '4.70', 'log2', 73, '3.57']    
 22) 72.83% [11101111110111101000000001000] [100, 18, 2, 2, '4.29', 'log2', 79, '2.24'] *  
 23) 72.83% [11101111110111101000000001000] [100, 18, 2, 2, '4.29', 'log2', 79, '2.24']    
 24) 72.82% [11111010110100100000000001000] [100, 15, 2, 1, '4.70', 'log2', 73, '3.57'] *  
 25) 72.80% [11101111110111101000000001010] [100, 16, 8, 1, '4.29', 'log2', 79, '2.24']    
 26) 72.77% [11111110110100100000000001010] [100, 16, 8, 1, '4.70', 'log2', 73, '3.57']    
 27) 72.77% [11111110110100111000000001010] [100, 18, 2, 1, '4.43', 'log2', 79, '2.24']    
 28) 72.77% [11111110110100111000000001010] [100, 18, 2, 1, '4.43', 'log2', 79, '2.24']    
 29) 72.75% [11111010110100100000001011000] [100, 15, 2, 1, '4.87', 'log2', 73, '3.57']    
 30) 72.75% [11101110110100100000000011010] [100, 16, 8, 1, '4.42', 'log2', 79, '2.62'] *  
 31) 72.73% [11111010110100100000000001010] [100, 14, 2, 1, '4.25', 'log2', 78, '2.62'] *  
 32) 72.73% [11111011110111101000000001001] [100, 18, 2, 2, '4.29', 'log2', 79, '2.24']    
 33) 72.73% [11111011110111101000000001001] [100, 18, 2, 2, '4.87', 'log2', 79, '2.24']    
 34) 72.73% [11111011110111101000000001001] [100, 18, 2, 2, '4.87', 'log2', 79, '2.24'] *  
 35) 72.72% [11101110110100100000001011000] [100, 15, 2, 1, '4.87', 'sqrt', 86, '2.86']    
 36) 72.72% [11101000110111100000000001010] [100, 16, 8, 1, '4.25', 'sqrt', 73, '3.57'] *  
 37) 72.72% [11101000110111100000000001010] [100, 16, 8, 1, '4.25', 'sqrt', 73, '3.57']    
 38) 72.72% [11111010110100100000000011010] [100, 14, 2, 1, '4.25', 'log2', 78, '2.62']    
 39) 72.72% [11111010110100100000000011010] [100, 14, 2, 1, '4.25', 'log2', 78, '2.62'] *  
 40) 72.72% [11111000110110101000000001001] [100, 18, 1, 1, '4.70', 'log2', 73, '3.57']    
 41) 72.72% [11111110110100100000000001010] [100, 18, 1, 2, '4.29', 'log2', 79, '2.24'] *  
 42) 72.72% [11111010110100101000000001001] [100, 18, 2, 2, '4.43', 'log2', 73, '2.24'] *  
 43) 72.71% [11111011101100100000000001010] [100, 16, 8, 1, '4.25', 'sqrt', 73, '3.57']    
 44) 72.71% [11111011101100100000000001010] [100, 16, 1, 1, '4.43', 'log2', 79, '3.57'] *  
 45) 72.71% [11111011110111101000000001010] [100, 18, 2, 2, '4.29', 'log2', 79, '2.24'] *  
 46) 72.71% [11101111110111101000000001010] [100, 18, 2, 2, '4.29', 'log2', 79, '2.24']    
 47) 72.71% [11101111110111101000000001010] [100, 18, 2, 2, '4.29', 'log2', 79, '2.24'] *  
 48) 72.71% [11111010110100100000000001001] [100, 18, 1, 1, '4.70', 'log2', 73, '3.57']    
 49) 72.71% [11111010110100100000000001001] [100, 18, 1, 1, '4.70', 'log2', 73, '3.57']    
 50) 72.71% [11111010110100100000000001001] [100, 18, 2, 1, '4.43', 'log2', 79, '2.24'] *  

==================  18 ==================
  1) 73.09% [11101111110111101000000001000] [100, 15, 2, 1, '4.87', 'sqrt', 86, '2.86']    
  2) 73.09% [11101111110111101000000001000] [100, 15, 2, 1, '4.87', 'sqrt', 86, '2.86']    
  3) 73.09% [11101111110111101000000001000] [100, 15, 2, 1, '4.70', 'log2', 73, '2.86'] *  
  4) 73.09% [11101111110111101000000001000] [100, 15, 2, 1, '4.87', 'sqrt', 86, '2.86'] *  
  5) 73.08% [11111011110110100000000001000] [100, 16, 8, 1, '4.29', 'log2', 78, '2.62'] *  
  6) 73.08% [11101111110110101000000001000] [100, 15, 2, 1, '4.70', 'log2', 73, '2.24'] *  
  7) 73.02% [11111011110111101000000001000] [100, 15, 2, 1, '4.29', 'log2', 79, '2.24']    
  8) 73.02% [11111011110111101000000001000] [100, 15, 2, 1, '4.70', 'log2', 79, '2.24'] *  
  9) 72.98% [11101111110110101000000001000] [100, 18, 2, 2, '4.29', 'log2', 79, '2.24']    
 10) 72.95% [11101011110100101000000001000] [100, 15, 2, 1, '4.70', 'log2', 73, '2.24'] *  
 11) 72.95% [11111110110100100000000001000] [100, 16, 8, 1, '4.87', 'sqrt', 78, '2.86'] *  
 12) 72.94% [11111110110100100000000001000] [100, 15, 2, 1, '4.87', 'sqrt', 86, '2.86']    
 13) 72.94% [11111110110100100000000001000] [100, 15, 2, 1, '4.87', 'sqrt', 78, '2.86']    
 14) 72.94% [11111110110100100000000001000] [100, 15, 2, 1, '4.87', 'sqrt', 86, '2.86']    
 15) 72.93% [11111111110111101000000001000] [100, 18, 1, 2, '4.29', 'log2', 79, '2.24'] *  
 16) 72.91% [11101011110111101000000001000] [100, 15, 2, 1, '4.70', 'log2', 73, '3.57']    
 17) 72.91% [11101011110111101000000001000] [100, 15, 2, 1, '4.70', 'log2', 73, '2.24']    
 18) 72.91% [11101011110111101000000001000] [100, 15, 2, 1, '4.70', 'log2', 73, '3.57']    
 19) 72.91% [11101011110111101000000001000] [100, 15, 2, 1, '4.70', 'log2', 73, '2.24'] *  
 20) 72.91% [11101011110111101000000001000] [100, 18, 2, 2, '4.29', 'log2', 79, '2.24'] *  
 21) 72.87% [11111011110111101000000001010] [100, 16, 8, 1, '4.29', 'log2', 79, '2.24']    
 22) 72.87% [11111011110111101000000001010] [100, 16, 8, 1, '4.42', 'log2', 79, '2.62']    
 23) 72.87% [11111011110111101000000001010] [100, 16, 8, 1, '4.29', 'log2', 78, '2.62']    
 24) 72.87% [11111011110111101000000001010] [100, 16, 2, 1, '4.29', 'log2', 79, '2.24']    
 25) 72.86% [11101110110100100000000001000] [100, 16, 8, 1, '4.42', 'log2', 79, '2.62']    
 26) 72.86% [11101110110100100000000001000] [100, 16, 8, 1, '4.42', 'log2', 79, '2.62']    
 27) 72.86% [11101110110100100000000001000] [100, 16, 8, 1, '4.42', 'log2', 79, '2.62']    
 28) 72.86% [11101110110100100000000001000] [100, 16, 8, 1, '4.42', 'log2', 79, '2.62'] *  
 29) 72.86% [11101110110100100000000001000] [100, 16, 8, 1, '4.42', 'log2', 79, '2.62'] *  
 30) 72.86% [11101011010111101000000001000] [100, 18, 1, 1, '4.70', 'log2', 73, '3.57']    
 31) 72.86% [11101110110100100000000001000] [100, 15, 2, 1, '4.42', 'log2', 79, '2.62']    
 32) 72.84% [11111000110111101000000001000] [100, 15, 2, 1, '4.87', 'sqrt', 73, '3.57']    
 33) 72.84% [11111011101100100000000001000] [100, 15, 2, 1, '4.70', 'log2', 73, '3.57']    
 34) 72.84% [11111011101100100000000001000] [100, 15, 2, 1, '4.87', 'sqrt', 86, '3.57'] *  
 35) 72.83% [11101111110111101000000001000] [100, 18, 2, 2, '4.29', 'log2', 79, '2.24']    
 36) 72.83% [11101111110111101000000001000] [100, 18, 2, 2, '4.29', 'log2', 79, '2.24']    
 37) 72.82% [11111010110100100000000001000] [100, 15, 2, 1, '4.70', 'log2', 73, '3.57']    
 38) 72.82% [11111010110100100000000001000] [100, 15, 2, 1, '4.25', 'log2', 78, '2.62'] *  
 39) 72.82% [11111010110100100000000001000] [100, 15, 2, 1, '4.29', 'log2', 79, '2.24'] *  
 40) 72.82% [11111010110100100000000001000] [100, 15, 2, 1, '4.25', 'log2', 78, '2.62'] *  
 41) 72.82% [11111010110100100000000001000] [100, 15, 2, 1, '4.70', 'log2', 73, '3.57'] *  
 42) 72.82% [11111011010100100000000001000] [100, 15, 2, 1, '4.70', 'log2', 73, '3.57'] *  
 43) 72.80% [11101111110111101000000001010] [100, 16, 8, 1, '4.29', 'log2', 79, '2.24']    
 44) 72.79% [11101000110111100000000001000] [100, 16, 8, 1, '4.25', 'sqrt', 73, '3.57'] *  
 45) 72.78% [11111110110100100000000011000] [100, 15, 2, 1, '4.87', 'sqrt', 78, '2.86'] *  
 46) 72.78% [11111011110111100000000001010] [100, 16, 8, 1, '4.29', 'log2', 79, '2.24'] *  
 47) 72.77% [11111010110100101000000001010] [100, 18, 2, 2, '4.29', 'log2', 79, '2.24'] *  
 48) 72.77% [11111110110100100000000001010] [100, 16, 8, 1, '4.70', 'log2', 73, '3.57']    
 49) 72.77% [11111110110100100000000001010] [100, 16, 8, 1, '4.70', 'log2', 73, '3.57'] *  
 50) 72.77% [11111011110111101000000001010] [100, 15, 2, 1, '4.29', 'log2', 79, '2.24'] *  

==================  19 ==================
  1) 73.16% [11101011110110101000000001000] [100, 15, 2, 1, '4.70', 'log2', 73, '2.24'] *  
  2) 73.09% [11101111110111101000000001000] [100, 15, 2, 1, '4.87', 'sqrt', 86, '2.86']    
  3) 73.09% [11101111110111101000000001000] [100, 15, 2, 1, '4.87', 'sqrt', 86, '2.86']    
  4) 73.09% [11101111110111101000000001000] [100, 15, 2, 1, '4.70', 'log2', 73, '2.86']    
  5) 73.09% [11101111110111101000000001000] [100, 15, 2, 1, '4.87', 'sqrt', 86, '2.86']    
  6) 73.09% [11101111110111101000000001000] [100, 15, 2, 1, '4.87', 'sqrt', 86, '2.86'] *  
  7) 73.09% [11101111110111101000000001000] [100, 15, 2, 1, '4.87', 'sqrt', 86, '3.57'] *  
  8) 73.09% [11101111110111101000000001000] [100, 15, 2, 1, '4.70', 'log2', 73, '2.86'] *  
  9) 73.09% [11101111110111101000000001000] [100, 15, 2, 1, '4.87', 'sqrt', 86, '2.86'] *  
 10) 73.09% [11101111110111101000000001000] [100, 15, 2, 1, '4.87', 'log2', 73, '3.57'] *  
 11) 73.09% [11101111110111101000000001000] [100, 15, 2, 1, '4.70', 'log2', 73, '2.24'] *  
 12) 73.08% [11111011110110100000000001000] [100, 16, 8, 1, '4.29', 'log2', 78, '2.62']    
 13) 73.08% [11101111110110101000000001000] [100, 15, 2, 1, '4.70', 'log2', 73, '2.24']    
 14) 73.07% [11111011010100101000000001000] [100, 15, 2, 1, '4.70', 'log2', 73, '3.57'] *  
 15) 73.02% [11111011110111101000000001000] [100, 15, 2, 1, '4.29', 'log2', 79, '2.24']    
 16) 73.02% [11111011110111101000000001000] [100, 15, 2, 1, '4.70', 'log2', 79, '2.24']    
 17) 73.02% [11111011110111101000000001000] [100, 15, 2, 1, '4.70', 'log2', 79, '2.24'] *  
 18) 72.98% [11101111110110101000000001000] [100, 18, 2, 2, '4.29', 'log2', 79, '2.24']    
 19) 72.96% [11111011110100100000000001000] [100, 16, 8, 1, '4.42', 'log2', 79, '2.62'] *  
 20) 72.95% [11101011110100101000000001000] [100, 15, 2, 1, '4.70', 'log2', 73, '2.24']    
 21) 72.95% [11111110110100100000000001000] [100, 16, 8, 1, '4.87', 'sqrt', 78, '2.86']    
 22) 72.94% [11111110110100100000000001000] [100, 15, 2, 1, '4.87', 'sqrt', 86, '2.86']    
 23) 72.94% [11111110110100100000000001000] [100, 15, 2, 1, '4.87', 'sqrt', 78, '2.86']    
 24) 72.94% [11111110110100100000000001000] [100, 15, 2, 1, '4.87', 'sqrt', 86, '2.86']    
 25) 72.94% [11111110110100100000000001000] [100, 15, 2, 1, '4.70', 'log2', 79, '2.24'] *  
 26) 72.93% [11111111110111101000000001000] [100, 18, 1, 2, '4.29', 'log2', 79, '2.24']    
 27) 72.92% [11111010110100100000000001000] [100, 16, 8, 1, '4.42', 'log2', 79, '2.62'] *  
 28) 72.92% [11101011110111101000000001000] [100, 16, 8, 1, '4.42', 'log2', 73, '3.57'] *  
 29) 72.91% [11101011110111101000000001000] [100, 15, 2, 1, '4.70', 'log2', 73, '3.57']    
 30) 72.91% [11101011110111101000000001000] [100, 15, 2, 1, '4.70', 'log2', 73, '2.24']    
 31) 72.91% [11101011110111101000000001000] [100, 15, 2, 1, '4.70', 'log2', 73, '3.57']    
 32) 72.91% [11101011110111101000000001000] [100, 15, 2, 1, '4.70', 'log2', 73, '2.24']    
 33) 72.91% [11101011110111101000000001000] [100, 15, 2, 1, '4.70', 'log2', 73, '2.24'] *  
 34) 72.91% [11101011110111101000000001000] [100, 15, 2, 1, '4.70', 'log2', 73, '2.24'] *  
 35) 72.91% [11101011110111101000000001000] [100, 15, 2, 1, '4.70', 'sqrt', 86, '2.86'] *  
 36) 72.91% [11101011110111101000000001000] [100, 15, 2, 1, '4.70', 'log2', 73, '2.24'] *  
 37) 72.91% [11101011110111101000000001000] [100, 15, 2, 1, '4.70', 'log2', 73, '3.57'] *  
 38) 72.91% [11101011110111101000000001000] [100, 18, 2, 2, '4.29', 'log2', 79, '2.24']    
 39) 72.89% [11111010110100100000000001000] [100, 14, 2, 1, '4.25', 'log2', 78, '2.62'] *  
 40) 72.89% [11111010110100100000000001000] [100, 14, 2, 1, '4.25', 'log2', 78, '2.62'] *  
 41) 72.89% [11111010110100100000000001000] [100, 14, 2, 1, '4.25', 'log2', 78, '2.62'] *  
 42) 72.87% [11111011110111101000000001010] [100, 16, 8, 1, '4.29', 'log2', 79, '2.24']    
 43) 72.87% [11111011110111101000000001010] [100, 16, 8, 1, '4.42', 'log2', 79, '2.62'] *  
 44) 72.87% [11111011110111101000000001010] [100, 16, 2, 1, '4.29', 'log2', 79, '2.24'] *  
 45) 72.87% [11111011110111101000000001010] [100, 16, 8, 1, '4.42', 'log2', 79, '2.62']    
 46) 72.87% [11111011110111101000000001010] [100, 16, 8, 1, '4.29', 'log2', 78, '2.62']    
 47) 72.87% [11111011110111101000000001010] [100, 16, 2, 1, '4.29', 'log2', 79, '2.24']    
 48) 72.86% [11101110110100100000000001000] [100, 16, 8, 1, '4.42', 'log2', 79, '2.62']    
 49) 72.86% [11101110110100100000000001000] [100, 16, 8, 1, '4.42', 'log2', 79, '2.62']    
 50) 72.86% [11101110110100100000000001000] [100, 16, 8, 1, '4.42', 'log2', 79, '2.62']    

==================  20 ==================
  1) 73.16% [11101011110110101000000001000] [100, 15, 2, 1, '4.70', 'log2', 73, '2.24']    
  2) 73.16% [11101011110110101000000001000] [100, 15, 2, 1, '4.42', 'log2', 79, '2.62'] *  
  3) 73.09% [11101111110111101000000001000] [100, 15, 2, 1, '4.87', 'sqrt', 86, '2.86']    
  4) 73.09% [11101111110111101000000001000] [100, 15, 2, 1, '4.87', 'sqrt', 86, '2.86']    
  5) 73.09% [11101111110111101000000001000] [100, 15, 2, 1, '4.70', 'log2', 73, '2.86']    
  6) 73.09% [11101111110111101000000001000] [100, 15, 2, 1, '4.87', 'sqrt', 86, '2.86']    
  7) 73.09% [11101111110111101000000001000] [100, 15, 2, 1, '4.87', 'sqrt', 86, '2.86']    
  8) 73.09% [11101111110111101000000001000] [100, 15, 2, 1, '4.87', 'sqrt', 86, '3.57']    
  9) 73.09% [11101111110111101000000001000] [100, 15, 2, 1, '4.70', 'log2', 73, '2.86']    
 10) 73.09% [11101111110111101000000001000] [100, 15, 2, 1, '4.87', 'sqrt', 86, '2.86']    
 11) 73.09% [11101111110111101000000001000] [100, 15, 2, 1, '4.87', 'log2', 73, '3.57']    
 12) 73.09% [11101111110111101000000001000] [100, 15, 2, 1, '4.70', 'log2', 73, '2.24']    
 13) 73.09% [11101111110111101000000001000] [100, 15, 2, 1, '4.87', 'sqrt', 86, '2.86'] *  
 14) 73.09% [11101111110111101000000001000] [100, 15, 2, 1, '4.87', 'sqrt', 86, '2.86'] *  
 15) 73.09% [11101111110111101000000001000] [100, 15, 2, 1, '4.87', 'sqrt', 86, '2.86'] *  
 16) 73.09% [11101111110111101000000001000] [100, 15, 2, 1, '4.87', 'sqrt', 86, '2.86'] *  
 17) 73.09% [11101111110111101000000001000] [100, 15, 2, 1, '4.87', 'sqrt', 86, '2.86'] *  
 18) 73.09% [11101111110111101000000001000] [100, 15, 2, 1, '4.87', 'sqrt', 86, '2.86'] *  
 19) 73.09% [11101111110111101000000001000] [100, 15, 2, 1, '4.70', 'log2', 73, '2.24'] *  
 20) 73.08% [11111011110110100000000001000] [100, 16, 8, 1, '4.29', 'log2', 78, '2.62']    
 21) 73.08% [11101111110110101000000001000] [100, 15, 2, 1, '4.70', 'log2', 73, '2.24']    
 22) 73.07% [11111011010100101000000001000] [100, 15, 2, 1, '4.70', 'log2', 73, '3.57']    
 23) 73.06% [11101111110110101000000001000] [100, 18, 2, 1, '4.42', 'log2', 79, '2.24'] *  
 24) 73.04% [11101011110100101000000001000] [100, 14, 2, 1, '4.70', 'log2', 73, '2.24'] *  
 25) 73.02% [11111011110111101000000001000] [100, 15, 2, 1, '4.29', 'log2', 79, '2.24']    
 26) 73.02% [11111011110111101000000001000] [100, 15, 2, 1, '4.70', 'log2', 79, '2.24']    
 27) 73.02% [11111011110111101000000001000] [100, 15, 2, 1, '4.70', 'log2', 79, '2.24']    
 28) 73.02% [11111011110111101000000001000] [100, 15, 2, 1, '4.70', 'log2', 79, '2.24'] *  
 29) 73.02% [11111011110111101000000001000] [100, 15, 2, 1, '4.29', 'log2', 79, '2.24'] *  
 30) 73.02% [11111011110111101000000001000] [100, 15, 2, 1, '4.29', 'log2', 79, '2.24'] *  
 31) 72.99% [11101110110110100000000001000] [100, 16, 8, 1, '4.42', 'log2', 79, '2.62'] *  
 32) 72.98% [11101111110110101000000001000] [100, 18, 2, 2, '4.29', 'log2', 79, '2.24']    
 33) 72.97% [11111011110100100000000001000] [100, 15, 2, 1, '4.42', 'log2', 79, '2.62'] *  
 34) 72.96% [11101111110100100000000001000] [100, 15, 2, 1, '4.87', 'sqrt', 86, '3.57'] *  
 35) 72.96% [11111011110100100000000001000] [100, 16, 8, 1, '4.42', 'log2', 79, '2.62']    
 36) 72.96% [11111011110100100000000001000] [100, 16, 8, 1, '4.42', 'log2', 79, '2.62'] *  
 37) 72.96% [11101011110100100000000001000] [100, 15, 2, 1, '4.87', 'sqrt', 78, '2.86'] *  
 38) 72.95% [11101011110100101000000001000] [100, 15, 2, 1, '4.70', 'log2', 73, '2.24']    
 39) 72.95% [11111110110100100000000001000] [100, 16, 8, 1, '4.87', 'sqrt', 78, '2.86']    
 40) 72.94% [11111110110100100000000001000] [100, 15, 2, 1, '4.87', 'sqrt', 86, '2.86']    
 41) 72.94% [11111110110100100000000001000] [100, 15, 2, 1, '4.87', 'sqrt', 78, '2.86']    
 42) 72.94% [11111110110100100000000001000] [100, 15, 2, 1, '4.87', 'sqrt', 86, '2.86']    
 43) 72.94% [11111110110100100000000001000] [100, 15, 2, 1, '4.70', 'log2', 79, '2.24']    
 44) 72.94% [11111110110100100000000001000] [100, 15, 2, 1, '4.70', 'log2', 79, '2.24'] *  
 45) 72.94% [11111110110100100000000001000] [100, 15, 2, 1, '4.87', 'sqrt', 86, '2.86'] *  
 46) 72.93% [11101111110111101000000001000] [100, 16, 8, 1, '4.42', 'log2', 79, '2.62'] *  
 47) 72.93% [11101111110111101000000001000] [100, 16, 8, 1, '4.87', 'sqrt', 86, '2.86'] *  
 48) 72.93% [11111111110111101000000001000] [100, 18, 1, 2, '4.29', 'log2', 79, '2.24']    
 49) 72.92% [11101111110110101000000001000] [100, 15, 2, 2, '4.29', 'log2', 79, '2.24'] *  
 50) 72.92% [11111010110100100000000001000] [100, 16, 8, 1, '4.42', 'log2', 79, '2.62']    

==================  21 ==================
  1) 73.16% [11101011110110101000000001000] [100, 15, 2, 1, '4.70', 'log2', 73, '2.24']    
  2) 73.16% [11101011110110101000000001000] [100, 15, 2, 1, '4.42', 'log2', 79, '2.62']    
  3) 73.16% [11101011110110101000000001000] [100, 15, 2, 1, '4.42', 'log2', 79, '2.62'] *  
  4) 73.09% [11101111110111101000000001000] [100, 15, 2, 1, '4.87', 'sqrt', 86, '2.86']    
  5) 73.09% [11101111110111101000000001000] [100, 15, 2, 1, '4.87', 'sqrt', 86, '2.86']    
  6) 73.09% [11101111110111101000000001000] [100, 15, 2, 1, '4.70', 'log2', 73, '2.86']    
  7) 73.09% [11101111110111101000000001000] [100, 15, 2, 1, '4.87', 'sqrt', 86, '2.86']    
  8) 73.09% [11101111110111101000000001000] [100, 15, 2, 1, '4.87', 'sqrt', 86, '2.86']    
  9) 73.09% [11101111110111101000000001000] [100, 15, 2, 1, '4.87', 'sqrt', 86, '3.57']    
 10) 73.09% [11101111110111101000000001000] [100, 15, 2, 1, '4.70', 'log2', 73, '2.86']    
 11) 73.09% [11101111110111101000000001000] [100, 15, 2, 1, '4.87', 'sqrt', 86, '2.86']    
 12) 73.09% [11101111110111101000000001000] [100, 15, 2, 1, '4.87', 'log2', 73, '3.57']    
 13) 73.09% [11101111110111101000000001000] [100, 15, 2, 1, '4.70', 'log2', 73, '2.24']    
 14) 73.09% [11101111110111101000000001000] [100, 15, 2, 1, '4.87', 'sqrt', 86, '2.86']    
 15) 73.09% [11101111110111101000000001000] [100, 15, 2, 1, '4.87', 'sqrt', 86, '2.86']    
 16) 73.09% [11101111110111101000000001000] [100, 15, 2, 1, '4.87', 'sqrt', 86, '2.86']    
 17) 73.09% [11101111110111101000000001000] [100, 15, 2, 1, '4.87', 'sqrt', 86, '2.86']    
 18) 73.09% [11101111110111101000000001000] [100, 15, 2, 1, '4.87', 'sqrt', 86, '2.86']    
 19) 73.09% [11101111110111101000000001000] [100, 15, 2, 1, '4.87', 'sqrt', 86, '2.86']    
 20) 73.09% [11101111110111101000000001000] [100, 15, 2, 1, '4.70', 'log2', 73, '2.24']    
 21) 73.09% [11101111110111101000000001000] [100, 15, 2, 1, '4.87', 'sqrt', 86, '2.86'] *  
 22) 73.09% [11101111110111101000000001000] [100, 15, 2, 1, '4.87', 'sqrt', 86, '2.86'] *  
 23) 73.09% [11101111110111101000000001000] [100, 15, 2, 1, '4.70', 'log2', 86, '2.86'] *  
 24) 73.09% [11101111110111101000000001000] [100, 15, 2, 1, '4.87', 'sqrt', 86, '2.86'] *  
 25) 73.09% [11101111110111101000000001000] [100, 15, 2, 1, '4.87', 'sqrt', 86, '2.86'] *  
 26) 73.09% [11101111110111101000000001000] [100, 15, 2, 1, '4.70', 'log2', 79, '2.24'] *  
 27) 73.09% [11101111110111101000000001000] [100, 15, 2, 1, '4.87', 'sqrt', 86, '2.86'] *  
 28) 73.09% [11101111110111101000000001000] [100, 15, 2, 1, '4.87', 'log2', 73, '3.57'] *  
 29) 73.09% [11101111110111101000000001000] [100, 15, 2, 1, '4.87', 'log2', 79, '2.24'] *  
 30) 73.09% [11101111110111101000000001000] [100, 15, 2, 1, '4.70', 'log2', 73, '2.86'] *  
 31) 73.09% [11101111110111101000000001000] [100, 15, 2, 1, '4.87', 'sqrt', 86, '2.86'] *  
 32) 73.09% [11101111110111101000000001000] [100, 15, 2, 1, '4.87', 'sqrt', 86, '3.57'] *  
 33) 73.09% [11101111110111101000000001000] [100, 15, 2, 1, '4.87', 'sqrt', 86, '2.86'] *  
 34) 73.08% [11111011110110100000000001000] [100, 16, 8, 1, '4.29', 'log2', 78, '2.62']    
 35) 73.08% [11101111110110101000000001000] [100, 15, 2, 1, '4.87', 'log2', 79, '2.24'] *  
 36) 73.08% [11101111110110101000000001000] [100, 15, 2, 1, '4.70', 'log2', 73, '2.24'] *  
 37) 73.08% [11101111110110101000000001000] [100, 15, 2, 1, '4.70', 'log2', 73, '2.24']    
 38) 73.08% [11101111110110101000000001000] [100, 15, 2, 1, '4.42', 'log2', 79, '2.24'] *  
 39) 73.07% [11111011010100101000000001000] [100, 15, 2, 1, '4.70', 'log2', 73, '3.57']    
 40) 73.07% [11111011010100101000000001000] [100, 15, 2, 1, '4.70', 'log2', 73, '3.57'] *  
 41) 73.06% [11101111110110101000000001000] [100, 18, 2, 1, '4.42', 'log2', 79, '2.24']    
 42) 73.04% [11101011110100101000000001000] [100, 14, 2, 1, '4.70', 'log2', 73, '2.24']    
 43) 73.02% [11101011110110100000000001000] [100, 15, 2, 1, '4.87', 'sqrt', 78, '2.86'] *  
 44) 73.02% [11111011110111101000000001000] [100, 15, 2, 1, '4.29', 'log2', 79, '2.24']    
 45) 73.02% [11111011110111101000000001000] [100, 15, 2, 1, '4.70', 'log2', 79, '2.24']    
 46) 73.02% [11111011110111101000000001000] [100, 15, 2, 1, '4.70', 'log2', 79, '2.24']    
 47) 73.02% [11111011110111101000000001000] [100, 15, 2, 1, '4.70', 'log2', 79, '2.24']    
 48) 73.02% [11111011110111101000000001000] [100, 15, 2, 1, '4.29', 'log2', 79, '2.24']    
 49) 73.02% [11111011110111101000000001000] [100, 15, 2, 1, '4.29', 'log2', 79, '2.24']    
 50) 73.02% [11111011110111101000000001000] [100, 15, 2, 1, '4.70', 'sqrt', 86, '2.86'] *  

==================  22 ==================
  1) 73.16% [11101011110110101000000001000] [100, 15, 2, 1, '4.70', 'log2', 73, '2.24']    
  2) 73.16% [11101011110110101000000001000] [100, 15, 2, 1, '4.42', 'log2', 79, '2.62']    
  3) 73.16% [11101011110110101000000001000] [100, 15, 2, 1, '4.42', 'log2', 79, '2.62']    
  4) 73.16% [11101011110110101000000001000] [100, 15, 2, 1, '4.70', 'log2', 73, '2.24'] *  
  5) 73.09% [11101111110111101000000001000] [100, 15, 2, 1, '4.87', 'sqrt', 86, '2.86']    
  6) 73.09% [11101111110111101000000001000] [100, 15, 2, 1, '4.87', 'sqrt', 86, '2.86']    
  7) 73.09% [11101111110111101000000001000] [100, 15, 2, 1, '4.70', 'log2', 73, '2.86']    
  8) 73.09% [11101111110111101000000001000] [100, 15, 2, 1, '4.87', 'sqrt', 86, '2.86']    
  9) 73.09% [11101111110111101000000001000] [100, 15, 2, 1, '4.87', 'sqrt', 86, '2.86']    
 10) 73.09% [11101111110111101000000001000] [100, 15, 2, 1, '4.87', 'sqrt', 86, '3.57']    
 11) 73.09% [11101111110111101000000001000] [100, 15, 2, 1, '4.70', 'log2', 73, '2.86']    
 12) 73.09% [11101111110111101000000001000] [100, 15, 2, 1, '4.87', 'sqrt', 86, '2.86']    
 13) 73.09% [11101111110111101000000001000] [100, 15, 2, 1, '4.87', 'log2', 73, '3.57']    
 14) 73.09% [11101111110111101000000001000] [100, 15, 2, 1, '4.70', 'log2', 73, '2.24']    
 15) 73.09% [11101111110111101000000001000] [100, 15, 2, 1, '4.87', 'sqrt', 86, '2.86']    
 16) 73.09% [11101111110111101000000001000] [100, 15, 2, 1, '4.87', 'sqrt', 86, '2.86']    
 17) 73.09% [11101111110111101000000001000] [100, 15, 2, 1, '4.87', 'sqrt', 86, '2.86']    
 18) 73.09% [11101111110111101000000001000] [100, 15, 2, 1, '4.87', 'sqrt', 86, '2.86']    
 19) 73.09% [11101111110111101000000001000] [100, 15, 2, 1, '4.87', 'sqrt', 86, '2.86']    
 20) 73.09% [11101111110111101000000001000] [100, 15, 2, 1, '4.87', 'sqrt', 86, '2.86']    
 21) 73.09% [11101111110111101000000001000] [100, 15, 2, 1, '4.70', 'log2', 73, '2.24']    
 22) 73.09% [11101111110111101000000001000] [100, 15, 2, 1, '4.87', 'sqrt', 86, '2.86']    
 23) 73.09% [11101111110111101000000001000] [100, 15, 2, 1, '4.87', 'sqrt', 86, '2.86']    
 24) 73.09% [11101111110111101000000001000] [100, 15, 2, 1, '4.70', 'log2', 86, '2.86']    
 25) 73.09% [11101111110111101000000001000] [100, 15, 2, 1, '4.87', 'sqrt', 86, '2.86']    
 26) 73.09% [11101111110111101000000001000] [100, 15, 2, 1, '4.87', 'sqrt', 86, '2.86']    
 27) 73.09% [11101111110111101000000001000] [100, 15, 2, 1, '4.70', 'log2', 79, '2.24']    
 28) 73.09% [11101111110111101000000001000] [100, 15, 2, 1, '4.87', 'sqrt', 86, '2.86']    
 29) 73.09% [11101111110111101000000001000] [100, 15, 2, 1, '4.87', 'log2', 73, '3.57']    
 30) 73.09% [11101111110111101000000001000] [100, 15, 2, 1, '4.87', 'log2', 79, '2.24']    
 31) 73.09% [11101111110111101000000001000] [100, 15, 2, 1, '4.70', 'log2', 73, '2.86']    
 32) 73.09% [11101111110111101000000001000] [100, 15, 2, 1, '4.87', 'sqrt', 86, '2.86']    
 33) 73.09% [11101111110111101000000001000] [100, 15, 2, 1, '4.87', 'sqrt', 86, '3.57']    
 34) 73.09% [11101111110111101000000001000] [100, 15, 2, 1, '4.87', 'sqrt', 86, '2.86']    
 35) 73.09% [11101111110111101000000001000] [100, 15, 2, 1, '4.70', 'log2', 73, '3.57'] *  
 36) 73.09% [11101111110111101000000001000] [100, 15, 2, 1, '4.87', 'sqrt', 86, '2.86'] *  
 37) 73.09% [11101111110111101000000001000] [100, 15, 2, 1, '4.70', 'log2', 73, '2.86'] *  
 38) 73.09% [11101111110111101000000001000] [100, 15, 2, 1, '4.87', 'sqrt', 86, '3.57'] *  
 39) 73.09% [11101111110111101000000001000] [100, 15, 2, 1, '4.87', 'sqrt', 86, '2.86'] *  
 40) 73.09% [11101111110111101000000001000] [100, 15, 2, 1, '4.87', 'sqrt', 86, '2.86'] *  
 41) 73.09% [11101111110111101000000001000] [100, 15, 2, 1, '4.87', 'sqrt', 86, '2.86'] *  
 42) 73.09% [11101111110111101000000001000] [100, 15, 2, 1, '4.87', 'sqrt', 86, '2.86'] *  
 43) 73.09% [11101111110111101000000001000] [100, 15, 2, 1, '4.87', 'sqrt', 86, '2.86'] *  
 44) 73.09% [11101111110111101000000001000] [100, 15, 2, 1, '4.70', 'log2', 73, '2.86'] *  
 45) 73.09% [11101111110111101000000001000] [100, 15, 2, 1, '4.87', 'sqrt', 86, '3.57'] *  
 46) 73.09% [11101111110111101000000001000] [100, 15, 2, 1, '4.70', 'log2', 73, '2.24'] *  
 47) 73.09% [11101111110111101000000001000] [100, 15, 2, 1, '4.87', 'sqrt', 86, '2.86'] *  
 48) 73.09% [11101111110111101000000001000] [100, 15, 2, 1, '4.87', 'sqrt', 86, '3.57'] *  
 49) 73.09% [11101111110111101000000001000] [100, 15, 2, 1, '4.87', 'sqrt', 86, '2.86'] *  
 50) 73.09% [11101111110111101000000001000] [100, 15, 2, 1, '4.87', 'sqrt', 86, '2.62'] *  

==================  23 ==================
  1) 73.16% [11101011110110101000000001000] [100, 15, 2, 1, '4.70', 'log2', 73, '2.24']    
  2) 73.16% [11101011110110101000000001000] [100, 15, 2, 1, '4.42', 'log2', 79, '2.62']    
  3) 73.16% [11101011110110101000000001000] [100, 15, 2, 1, '4.42', 'log2', 79, '2.62']    
  4) 73.16% [11101011110110101000000001000] [100, 15, 2, 1, '4.70', 'log2', 73, '2.24']    
  5) 73.16% [11101011110110101000000001000] [100, 15, 2, 1, '4.42', 'log2', 79, '2.86'] *  
  6) 73.16% [11101011110110101000000001000] [100, 15, 2, 1, '4.42', 'log2', 79, '2.62'] *  
  7) 73.16% [11101011110110101000000001000] [100, 15, 2, 1, '4.87', 'log2', 73, '2.24'] *  
  8) 73.16% [11101011110110101000000001000] [100, 15, 2, 1, '4.70', 'log2', 73, '2.24'] *  
  9) 73.16% [11101011110110101000000001000] [100, 15, 2, 1, '4.70', 'log2', 73, '2.24'] *  
 10) 73.09% [11101111110111101000000001000] [100, 15, 2, 1, '4.87', 'sqrt', 86, '2.86']    
 11) 73.09% [11101111110111101000000001000] [100, 15, 2, 1, '4.87', 'sqrt', 86, '2.86']    
 12) 73.09% [11101111110111101000000001000] [100, 15, 2, 1, '4.70', 'log2', 73, '2.86']    
 13) 73.09% [11101111110111101000000001000] [100, 15, 2, 1, '4.87', 'sqrt', 86, '2.86']    
 14) 73.09% [11101111110111101000000001000] [100, 15, 2, 1, '4.87', 'sqrt', 86, '2.86']    
 15) 73.09% [11101111110111101000000001000] [100, 15, 2, 1, '4.87', 'sqrt', 86, '3.57']    
 16) 73.09% [11101111110111101000000001000] [100, 15, 2, 1, '4.70', 'log2', 73, '2.86']    
 17) 73.09% [11101111110111101000000001000] [100, 15, 2, 1, '4.87', 'sqrt', 86, '2.86']    
 18) 73.09% [11101111110111101000000001000] [100, 15, 2, 1, '4.87', 'log2', 73, '3.57']    
 19) 73.09% [11101111110111101000000001000] [100, 15, 2, 1, '4.70', 'log2', 73, '2.24']    
 20) 73.09% [11101111110111101000000001000] [100, 15, 2, 1, '4.87', 'sqrt', 86, '2.86']    
 21) 73.09% [11101111110111101000000001000] [100, 15, 2, 1, '4.87', 'sqrt', 86, '2.86']    
 22) 73.09% [11101111110111101000000001000] [100, 15, 2, 1, '4.87', 'sqrt', 86, '2.86']    
 23) 73.09% [11101111110111101000000001000] [100, 15, 2, 1, '4.87', 'sqrt', 86, '2.86']    
 24) 73.09% [11101111110111101000000001000] [100, 15, 2, 1, '4.87', 'sqrt', 86, '2.86']    
 25) 73.09% [11101111110111101000000001000] [100, 15, 2, 1, '4.87', 'sqrt', 86, '2.86']    
 26) 73.09% [11101111110111101000000001000] [100, 15, 2, 1, '4.70', 'log2', 73, '2.24']    
 27) 73.09% [11101111110111101000000001000] [100, 15, 2, 1, '4.87', 'sqrt', 86, '2.86']    
 28) 73.09% [11101111110111101000000001000] [100, 15, 2, 1, '4.87', 'sqrt', 86, '2.86']    
 29) 73.09% [11101111110111101000000001000] [100, 15, 2, 1, '4.70', 'log2', 86, '2.86']    
 30) 73.09% [11101111110111101000000001000] [100, 15, 2, 1, '4.87', 'sqrt', 86, '2.86']    
 31) 73.09% [11101111110111101000000001000] [100, 15, 2, 1, '4.87', 'sqrt', 86, '2.86']    
 32) 73.09% [11101111110111101000000001000] [100, 15, 2, 1, '4.70', 'log2', 79, '2.24']    
 33) 73.09% [11101111110111101000000001000] [100, 15, 2, 1, '4.87', 'sqrt', 86, '2.86']    
 34) 73.09% [11101111110111101000000001000] [100, 15, 2, 1, '4.87', 'log2', 73, '3.57']    
 35) 73.09% [11101111110111101000000001000] [100, 15, 2, 1, '4.87', 'log2', 79, '2.24']    
 36) 73.09% [11101111110111101000000001000] [100, 15, 2, 1, '4.70', 'log2', 73, '2.86']    
 37) 73.09% [11101111110111101000000001000] [100, 15, 2, 1, '4.87', 'sqrt', 86, '2.86']    
 38) 73.09% [11101111110111101000000001000] [100, 15, 2, 1, '4.87', 'sqrt', 86, '3.57']    
 39) 73.09% [11101111110111101000000001000] [100, 15, 2, 1, '4.87', 'sqrt', 86, '2.86']    
 40) 73.09% [11101111110111101000000001000] [100, 15, 2, 1, '4.70', 'log2', 73, '3.57']    
 41) 73.09% [11101111110111101000000001000] [100, 15, 2, 1, '4.87', 'sqrt', 86, '2.86']    
 42) 73.09% [11101111110111101000000001000] [100, 15, 2, 1, '4.70', 'log2', 73, '2.86']    
 43) 73.09% [11101111110111101000000001000] [100, 15, 2, 1, '4.87', 'sqrt', 86, '3.57']    
 44) 73.09% [11101111110111101000000001000] [100, 15, 2, 1, '4.87', 'sqrt', 86, '2.86']    
 45) 73.09% [11101111110111101000000001000] [100, 15, 2, 1, '4.87', 'sqrt', 86, '2.86']    
 46) 73.09% [11101111110111101000000001000] [100, 15, 2, 1, '4.87', 'sqrt', 86, '2.86']    
 47) 73.09% [11101111110111101000000001000] [100, 15, 2, 1, '4.87', 'sqrt', 86, '2.86']    
 48) 73.09% [11101111110111101000000001000] [100, 15, 2, 1, '4.87', 'sqrt', 86, '2.86']    
 49) 73.09% [11101111110111101000000001000] [100, 15, 2, 1, '4.70', 'log2', 73, '2.86']    
 50) 73.09% [11101111110111101000000001000] [100, 15, 2, 1, '4.87', 'sqrt', 86, '3.57']    

==================  24 ==================
  1) 73.16% [11101011110110101000000001000] [100, 15, 2, 1, '4.70', 'log2', 73, '2.24']    
  2) 73.16% [11101011110110101000000001000] [100, 15, 2, 1, '4.42', 'log2', 79, '2.62']    
  3) 73.16% [11101011110110101000000001000] [100, 15, 2, 1, '4.42', 'log2', 79, '2.62']    
  4) 73.16% [11101011110110101000000001000] [100, 15, 2, 1, '4.70', 'log2', 73, '2.24']    
  5) 73.16% [11101011110110101000000001000] [100, 15, 2, 1, '4.42', 'log2', 79, '2.86']    
  6) 73.16% [11101011110110101000000001000] [100, 15, 2, 1, '4.42', 'log2', 79, '2.62']    
  7) 73.16% [11101011110110101000000001000] [100, 15, 2, 1, '4.87', 'log2', 73, '2.24']    
  8) 73.16% [11101011110110101000000001000] [100, 15, 2, 1, '4.70', 'log2', 73, '2.24']    
  9) 73.16% [11101011110110101000000001000] [100, 15, 2, 1, '4.70', 'log2', 73, '2.24']    
 10) 73.16% [11101011110110101000000001000] [100, 15, 2, 1, '4.70', 'log2', 73, '2.24'] *  
 11) 73.16% [11101011110110101000000001000] [100, 15, 2, 1, '4.70', 'log2', 73, '2.86'] *  
 12) 73.16% [11101011110110101000000001000] [100, 15, 2, 1, '4.70', 'sqrt', 86, '2.24'] *  
 13) 73.16% [11101011110110101000000001000] [100, 15, 2, 1, '4.87', 'log2', 73, '3.57'] *  
 14) 73.09% [11101111110111101000000001000] [100, 15, 2, 1, '4.87', 'sqrt', 86, '2.86']    
 15) 73.09% [11101111110111101000000001000] [100, 15, 2, 1, '4.87', 'sqrt', 86, '2.86']    
 16) 73.09% [11101111110111101000000001000] [100, 15, 2, 1, '4.70', 'log2', 73, '2.86']    
 17) 73.09% [11101111110111101000000001000] [100, 15, 2, 1, '4.87', 'sqrt', 86, '2.86']    
 18) 73.09% [11101111110111101000000001000] [100, 15, 2, 1, '4.87', 'sqrt', 86, '2.86']    
 19) 73.09% [11101111110111101000000001000] [100, 15, 2, 1, '4.87', 'sqrt', 86, '3.57']    
 20) 73.09% [11101111110111101000000001000] [100, 15, 2, 1, '4.70', 'log2', 73, '2.86']    
 21) 73.09% [11101111110111101000000001000] [100, 15, 2, 1, '4.87', 'sqrt', 86, '2.86']    
 22) 73.09% [11101111110111101000000001000] [100, 15, 2, 1, '4.87', 'log2', 73, '3.57']    
 23) 73.09% [11101111110111101000000001000] [100, 15, 2, 1, '4.70', 'log2', 73, '2.24']    
 24) 73.09% [11101111110111101000000001000] [100, 15, 2, 1, '4.87', 'sqrt', 86, '2.86']    
 25) 73.09% [11101111110111101000000001000] [100, 15, 2, 1, '4.87', 'sqrt', 86, '2.86']    
 26) 73.09% [11101111110111101000000001000] [100, 15, 2, 1, '4.87', 'sqrt', 86, '2.86']    
 27) 73.09% [11101111110111101000000001000] [100, 15, 2, 1, '4.87', 'sqrt', 86, '2.86']    
 28) 73.09% [11101111110111101000000001000] [100, 15, 2, 1, '4.87', 'sqrt', 86, '2.86']    
 29) 73.09% [11101111110111101000000001000] [100, 15, 2, 1, '4.87', 'sqrt', 86, '2.86']    
 30) 73.09% [11101111110111101000000001000] [100, 15, 2, 1, '4.70', 'log2', 73, '2.24']    
 31) 73.09% [11101111110111101000000001000] [100, 15, 2, 1, '4.87', 'sqrt', 86, '2.86']    
 32) 73.09% [11101111110111101000000001000] [100, 15, 2, 1, '4.87', 'sqrt', 86, '2.86']    
 33) 73.09% [11101111110111101000000001000] [100, 15, 2, 1, '4.70', 'log2', 86, '2.86']    
 34) 73.09% [11101111110111101000000001000] [100, 15, 2, 1, '4.87', 'sqrt', 86, '2.86']    
 35) 73.09% [11101111110111101000000001000] [100, 15, 2, 1, '4.87', 'sqrt', 86, '2.86']    
 36) 73.09% [11101111110111101000000001000] [100, 15, 2, 1, '4.70', 'log2', 79, '2.24']    
 37) 73.09% [11101111110111101000000001000] [100, 15, 2, 1, '4.87', 'sqrt', 86, '2.86']    
 38) 73.09% [11101111110111101000000001000] [100, 15, 2, 1, '4.87', 'log2', 73, '3.57']    
 39) 73.09% [11101111110111101000000001000] [100, 15, 2, 1, '4.87', 'log2', 79, '2.24']    
 40) 73.09% [11101111110111101000000001000] [100, 15, 2, 1, '4.70', 'log2', 73, '2.86']    
 41) 73.09% [11101111110111101000000001000] [100, 15, 2, 1, '4.87', 'sqrt', 86, '2.86']    
 42) 73.09% [11101111110111101000000001000] [100, 15, 2, 1, '4.87', 'sqrt', 86, '3.57']    
 43) 73.09% [11101111110111101000000001000] [100, 15, 2, 1, '4.87', 'sqrt', 86, '2.86']    
 44) 73.09% [11101111110111101000000001000] [100, 15, 2, 1, '4.70', 'log2', 73, '3.57']    
 45) 73.09% [11101111110111101000000001000] [100, 15, 2, 1, '4.87', 'sqrt', 86, '2.86']    
 46) 73.09% [11101111110111101000000001000] [100, 15, 2, 1, '4.70', 'log2', 73, '2.86']    
 47) 73.09% [11101111110111101000000001000] [100, 15, 2, 1, '4.87', 'sqrt', 86, '3.57']    
 48) 73.09% [11101111110111101000000001000] [100, 15, 2, 1, '4.87', 'sqrt', 86, '2.86']    
 49) 73.09% [11101111110111101000000001000] [100, 15, 2, 1, '4.87', 'sqrt', 86, '2.86']    
 50) 73.09% [11101111110111101000000001000] [100, 15, 2, 1, '4.87', 'sqrt', 86, '2.86']    


In [ ]:
feats_ = filt([1,1,1,0,1,0,1,1,1,1,0,1,1,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0])
feats_

In [ ]:
trains = []
valids = []

for i in range(20):
    t, v = train_test_split(data, test_size=0.3, random_state=i+306)
    t = prepare_data(t, t)
    v = prepare_data(v, t)
    trains.append(t)
    valids.append(v)

In [ ]:
for i in range(100):
    model = RF(n_estimators=1111,
               max_depth = 15,
               min_samples_leaf = 1,
               max_features = 'log2',
               random_state=42,
               n_jobs=-1)
    model.fit(trains[i].loc[:, fts], np.array(trains[i]['Price']))
    pred_valid = model.predict(valids[i].loc[:, fts])
    print(r2(valids[i]['Price'], pred_valid))